<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Libraires And Utilities</h1>


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
from tqdm.notebook import tqdm
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as ex
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
pyo.init_notebook_mode()
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('vader_lexicon')
from sklearn.cluster import KMeans
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import random
%pip install pyvis

plt.rc('figure',figsize=(17,13))

In [ ]:
b_df = pd.read_csv('/kaggle/input/billboard-the-hot-100-songs/charts.csv')
b_df.date = pd.to_datetime(b_df.date)
b_df.head(3)

In [ ]:
#extract date components
b_df = b_df.assign(year = b_df.date.dt.year)
b_df = b_df.assign(week = b_df.date.dt.isocalendar().week)
b_df = b_df.assign(month = b_df.date.dt.month)
b_df.drop(index=b_df.query('year == 2021').index,inplace=True)


<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Exploratory Data Analysis</h1>


<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Song Analysis</h1>


In [ ]:
by_year = b_df.groupby("year")

by_year = by_year["song"].agg(lambda column: "%".join(column)).to_frame().reset_index()


n_song_year                                                 = []
n1_song_name,n2_song_name,n3_song_name                      = [],[],[]
n1_song_appearances,n2_song_appearances,n3_song_appearances = [],[],[]

for year,songs in by_year.iterrows():
    top = nltk.FreqDist(songs['song'].split('%')).most_common(3)
    
    n_song_year.append(songs.year)
    n1_song_name.append(top[0][0])
    n2_song_name.append(top[1][0])
    n3_song_name.append(top[2][0])
    n1_song_appearances.append(top[0][1])
    n2_song_appearances.append(top[1][1])
    n3_song_appearances.append(top[2][1])

    
fig = go.Figure()

n1 = go.Scatter(x=n_song_year,y=n1_song_appearances,mode='lines+markers',text =[f"Song Name: <b>{i}<b>" for i in n1_song_name],showlegend=True,name='<b>№ 1 Song<b>')  
n2 = go.Scatter(x=n_song_year,y=n2_song_appearances,mode='lines+markers',text =[f"Song Name: <b>{i}<b>" for i in n2_song_name],showlegend=True,name='<b>№ 2 Song<b>')  
n3 = go.Scatter(x=n_song_year,y=n3_song_appearances,mode='lines+markers',text =[f"Song Name: <b>{i}<b>" for i in n3_song_name],showlegend=True,name='<b>№ 3 Song<b>')


fig.add_trace(n1)
fig.add_trace(n2)
fig.add_trace(n3)

fig.update_layout(title='<b>Yearly Top Song and The Number of Appearances on Lists<b>',xaxis_title='<b>Year<b>',yaxis_title='<b>Numer of Appearances<b>')
fig.update_layout(hovermode="x unified")

fig.show()


**Observation**: When looking at the number of times a song appeared on the Billboard top 100 list in a particular year, we can deduce that the most impactful and popular song has the largest number of appearances.
In the figure above, we can observe two important points; the first is the trend starting roughly around 1985 where the number 1 song, i.e., the one with the most appearances, is becoming more and more obsolete with time, meaning we each year we the listeners prefer a particular song and our preference stays stationary over the list posted through the year.
And the second observation which we will further analyze is the difference between the ranks, i.e., how different are the number 1,2 and 3 places in terms of appearances, a small assumption to we make goes as following; We consider a song to be outstandingly popular and catchy in the case where the difference between the 1 and 2/3 rank is at its highest.


In [ ]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=('<b>Difference Between № 1 and № 2 in a Given Year<b>',  '<b>Difference Between № 2 and № 3 in a Given Year<b>'))
n1_np = np.array(n1_song_appearances)
n2_np = np.array(n2_song_appearances)
n3_np = np.array(n3_song_appearances)

fig.add_trace(
    go.Scatter(x=n_song_year, y=np.abs(n1_np-n2_np),name='Difference'),
    row=1, col=1
)
fig.add_shape(type="line",
    x0=n_song_year[0], y0=np.abs(n1_np-n2_np).mean(), x1=n_song_year[-1], y1=np.abs(n1_np-n2_np).mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x1', 
        yref='y1')

fig.add_trace(
    go.Scatter(x=n_song_year, y=np.abs(n2_np-n3_np),name='Difference'),
    row=2, col=1
)
fig.add_shape(type="line",
    x0=n_song_year[0], y0=np.abs(n2_np-n3_np).mean(), x1=n_song_year[-1], y1=np.abs(n2_np-n3_np).mean(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x1', 
        yref='y2')


fig['layout']['xaxis2']['title'] = 'Date'
fig['layout']['yaxis2']['title'] = 'Appearances Difference'
fig['layout']['yaxis1']['title'] = 'Appearances Difference'

fig.update_layout(title_text="Difference Between Song Ranks")
fig.update_layout(hovermode="x unified")

fig.show()

In [ ]:
R = pd.DataFrame({'Artist':np.array(n1_song_name)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()],
'Appearance':np.array(n1_song_appearances)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()],
              'Year':np.array(n_song_year)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()]})

outstanding_songs = R.copy()
fig = go.Figure()
fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Artist<b>',"<b>Appearance<b>",'<b>Year<b>'],
            font=dict(size=19,family="Lato"),
            align="center"
        ),
        cells=dict(
            values=[R[k].tolist() for k in ['Artist',"Appearance",'Year']],
            align = "center")
    ),
    
)

fig.update_layout(title='List of Songs that Were the Most Significantly Different From 1 and 2 Place')
fig.show()

**Observation**: After looking at the above figure and observing the spikes in the differences between the first rank and the second rank, we defined a song as being outstandingly impactful by asking is it above the average difference plus one standard deviation.
The above table shows the rank one song names and the year they held a significant difference from rank two.
Note that the magnitude of difference between rank one song and rank two songs is significantly larger on average than between rank two and three.
Interestingly, if we look carefully at the years, we can see that these unique songs are distributed relatively uniformly around our timeline, and there is no particular period where more songs matched our definition. 


<a id="1"></a>

<h1 style="background-color:z;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Adding Lyrics of № 1 Songs </h1>


In [ ]:
lyrics_of_top_song_n1 = [ 
    "You cheated, you lied, You said that you love me You cheated, you lied You said that you want me Oh, what can I do but just keep on loving you? Love is something you know nothing about Love is somethng you can't throw about So please try to love me Love me like I love you Oh, please try to love me Love me like I love you You cheated, you lied You said that you want me You cheated, you lied, You said that you love me Oh, what can I do? Oh, what can I do? Oh, what can I do?",
    "Mona lisa, mona lisa, men have named you You're so like the lady with the mystic smile Is it only 'cause you're lonely they have blamed you? For that mona lisa strangeness in your smile? Do you smile to tempt a lover, mona lisa? Or is this your way to hide a broken heart? Many dreams have been brought to your doorstep They just lie there and they die there Are you warm, are you real, mona lisa? Or just a cold and lonely lovely work of art? Do you smile to tempt a lover, mona lisa? Or is this your way to hide a broken heart? Many dreams have been brought to your doorstep They just lie there and they die there Are you warm, are you real, mona lisa? Or just a cold and lonely lovely work of art?",
    "When life doesn't seem worth the living And you don't really care who you are When you feel there is no one beside you Look for a star When you know you're alone and so lonely And your friends have traveled afar There is someone waiting to guide you Look for a star Oh everyone has a lucky star That shines in the sky up above If you wish on a lucky star You're sure to find someone to love A rich man man, a poor man, a beggar No matter whoever you are There's a friend who's waiting to guide you Look for a star A rich man, a poor man, a beggar No matter whoever you are There's a friend who's waiting to guide you Look for a star.",
    "This land is mine, God gave this land to me This brave and ancient land to me And when the morning sun reveals her hills and plain Then I see a land where children can run free So take my hand and walk this land with me And walk this lovely land with me Though I am just a man, when you are by my side With the help of God, I know I can be strong Though I am just a man, when you are by my side With the help of God, I know I can be strong To make this land our home If I must fight, I'll fight to make this land our own Until I die, this land is mine",
    "Every limbo boy and girl All around the limbo world Gonna do the limbo rock All around the limbo clock Jack be limbo, Jack be quick Jack go unda limbo stick All around the limbo clock Hey, let's do the limbo rock Limbo lower now Limbo lower now How low can you go? First you spread your limbo feet Then you move to limbo beat Limbo ankolimboneee Bend back like a limbo tree Jack be limbo, Jack be quick Jack go unda limbo stick All around the limbo clock Hey, let's do the limbo rock La, la, la La, la, la La, la, la Get yourself a limbo girl Give that chic a limbo whirl There's a limbo moon above You will fall in limbo love Jack be limbo, Jack be quick Jack go unda limbo stick All around the limbo clock Hey, let's do the limbo rock Don't move that limbo bar You'll be a limbo star How low can you go?",
    "The days of wine and roses laugh and run away like a child at play Through a meadow land toward a closing door A door marked nevermore that wasn't there before The lonely night discloses just a passing breeze filled with memories Of the golden smile that introduced me to The days of wine and roses and you (The lonely night discloses) just a passing breeze filled with memories Of the golden smile that introduced me to The days of wine and roses and you-oo-oo",
    "Hello Dolly This is Louis, Dolly It's so nice to have you back where you belong You're lookin' swell, Dolly I can tell, Dolly You're still glowin', you're still crowin', you're still goin' strong I feel that room swayin' while the band's playin' One of your old favourite songs from way back when So take her wrap, fellas find her an empty lap, fellas Dolly'll, never go away again I feel the room swayin' while the band's playin' One of your old favourite songs from way back when So golly, gee, fellas have a little faith in me, fellas Dolly'll, never go away, promise you never go away Dolly'll, never go away again",
    "I want some red roses for a blue lady Mister florist take my order please We had a silly quarrel the other day I hope these pretty flowers chase her blues away I want some red roses for a blue lady Send them to the sweetest gal in town And if they do the trick, I'll hurry back to pick Your best white orchid for her wedding gown We had a silly quarrel the other day I hope these pretty flowers chase her blues away I want some red roses for a blue, blue lady Send them to the sweetest gal in town And if they do the trick, I'll hurry back to pick Your best white orchid for her wedding gown Your best white orchid for her wedding gown",
    "Michelle ma belle These are words that go together well My michelle Michelle ma belle Sont les mots qui vont tres bien ensemble Tres bien ensemble I love you, I love you, I love you Thats all I want to say Until I find a way I will say the only words I know that you'll understand Michelle ma belle Sont les mots qui vont tres bien ensemble Tres bien ensemble I need to, I need to, I need to I need to make you see Oh what you mean to me Until I do I'm hoping you will know what I mean I love you I want you, I want you, I want you I think you know by now Ill get to you somehow Until I do I'm telling you so youll understand Michelle ma belle Sont les mots qui vont tres bien ensemble Tres bien ensemble And I will say the only words I know that you'll understand My michelle",
    "You know, sometimes we're not prepared for adversity. When it happens sometimes, we're caught short. We don't know exactly how to handle it when it comes up. Sometimes, we don't know just what to do when adversity takes over. (chuckle). And I have advice for all of us, I got it from my pianist Joe Zawinul who wrote this tune. And it sounds like what you're supposed to say when you have that kind of problem. It's called mercy, mercy, mercy.",
    "And I wake up in the mornin With my hair down in my eyes and she says Hi And I stumble to the breakfast table While the kids are goin off to schoo, goodbye And she reaches out n takes my hand And squeezes it n says How ya feelin, hon? And I look across at smilin lips That warm my heart and see my mornin sun And if thats not lovin me Then all Ive got to say God didnt make little green apples And it dont rain in Indianapolis in the summertime And theres no such thing as Doctor Seuss Or Disneyland, and Mother Goose, no nursery rhyme God didnt make little green apples And it dont rain in Indianapolis in the summertime And when my self is feelin low I think about her face aglow and ease my mind Sometimes I call her up at home knowin shes busy And ask her if she could get away and meet me Maybe we could grab a bite to eat And she drops what shes doin and she hurries down to meet me And Im always late But she sits waitin patiently and smiles when she first sees me Cause shes made that way And if that aint lovin me Then all Ive got to say God didnt make little green apples And it dont snow in Minneapolis when the winter comes And theres no such thing as make-believe Puppy dogs, autumn leaves n BB guns...",
    "Left a good job in the city, Workin' for the man ev'ry night and day, And I never lost one minute of sleepin', Worryin' 'bout the way things might have been. Big wheel keep on turnin', Proud Mary keep on burnin', Rollin', rollin', rollin' on the river. Cleaned a lot of plates in Memphis, Pumped a lot of 'pane down in New Orleans, But I never saw the good side of the city, 'Til I hitched a ride on a river boat queen. Big wheel keep on turnin', Proud Mary keep on burnin', Rollin', rollin', rollin' on the river. Rollin', rollin', rollin' on the river. If you come down to the river, Bet you gonna find some people who live. You don't have to worry 'cause you have no money, People on the river are happy to give. Big wheel keep on turnin', Proud Mary keep on burnin', Rollin', rollin', rollin' on the river. Rollin', rollin', rollin' on the river. Rollin', rollin', rollin' on the river. Rollin', rollin', rollin' on the river.",
    "Hey, hey, hey, hey Beat is gettin' stronger Music gettin' longer, too Music is a-flashin' me I want to, I want to, I want to take you higher I wanna take you higher Baby, baby, baby, light my fire I wanna take you higher Boom laka-laka-laka, Boom laka-lak-goon-ka boom Hey, hey, hey, hey Beat is nitty-gritty Sound is in your city, too Music's still flashin' me Don't ya, don't ya, don't, don't, don't ya wanna get higher? Don't ya wanna get higher? Baby, baby, baby, light my fire (Woo) I wanna take you higher Boom laka-laka-laka, Boom laka-lak-goon-ka boom Hey, hey, hey, hey Boom laka-laka-laka, Boom laka-lak-goon-ka boom Higher Higher Higher Higher Higher (Won't ya light my fire?) Higher (Woo, yeah) Higher (Wanna take you higher) Higher Hey, hey, hey, hey Beat is there to make you move Sound is there to help you groove Music's still flashin' me Take your places I wanna take you higher (Higher) Wanna take you higher Baby, baby, baby, light my fire (Woo) I wanna take you high, high, high, high, high, high, high, high Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom Wanna take you, do you wanna go Wanna take my fire Wanna take you higher Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom, Boom laka-laka-laka, Boom laka-lak-goon-ka boom (Higher) Don't ya wanna get higher? I wanna take you higher Hey, hey, hey, hey Boom laka-laka-laka, Boom laka-lak-goon-ka boom I wanna take you higher I wanna take you higher Boom laka-laka-laka, Boom laka-lak-goon-ka boom Everybody higher, higher, higher",
    "I don't know how to love him What to do, how to move him I've been changed, yes, really changed In these past few days when I've seen myself I seem like someone else I don't know how to take this I don't see why he moves me He's a man, he's just a man And I've had so many men before In very many ways He's just one more Should I bring him down? Should I scream and shout? Should I speak of love? Let my feelings out? I never thought I'd come to this What's it all about? Don't you think it's rather funny I should be in this position? I'm the one who's always been So calm, so cool, no lover's fool Running every show He scares me so I never thought I'd come to this What's it all about? What's it all about? If he said he loved me I'd be lost, I'd be frightened I couldn't cope, just couldn't cope I'd turn my head, I'd back away I wouldn't want to know He scares me so Oh, I want him so I love him so",
    "Ah Well, well, well, well, well, well, well Wow, wow, wow (hoo hoo hoo) I was slippin' into darkness When they took my friend away I was slippin' into darkness When they took, when they took my friend away You know he loves to drink good whiskey (Wo ho ho ho) While laughing at the moon Slippin' into darkness Take my mind beyond the dreams I was slippin' into darkness, yeah Take my mind beyond the dreams Where I talk to my brother (Wo ho ho ho) Who never said their name Slippin' into darkness, yeah All my trouble so I choose I was slippin' into darkness, yeah All my trouble so I choose I got a wife and a baby, yeah yeah (Oh ho ho ho) Now my love hath gained its fame, yeah Slippin' into darkness, yeah When I heard my mother say I was slippin' into darkness When I heard my mother say Hey, what'd she say what'd she say You've been slippin' into darkness (Wo ho ho ho) Pretty soon you gonna pay, hey Ooh, ooh, ooh, ooh Ooh, ooh, ooh",
    "Why me Lord, what have I ever done To deserve even one Of the pleasures I've known Tell me Lord, what did I ever do That was worth loving you Or the kindness you've shown. Lord help me Jesus, I've wasted it so Help me Jesus I know what I am Now that I know that I've need you so Help me Jesus, my soul's in your hand. Tell me Lord, if you think there's a way I can try to repay All I've taken from you Maybe Lord, I can show someone else What I've been through myself On my way back to you. Lord help me Jesus, I've wasted it so Help me Jesus I know what I am Now that I know that I've need you so Help me Jesus, my soul's in your hand.",
    "She's a woman She's a baby She's a witch She's a lady She's a free and gentle spirit She can be what she wants to be And if she wants to be with me tonight That's alright She's one hell of a woman Young, strong and tame But soft when she loves me Like a gentle kitten in my hand She makes me feel Like a hell of a man She's wild, warm and tender She'll resist then she'll surrender She's a free and gentle spirit She can be what she wants to be And if she wants to be with me tonight Well that's alright She's one hell of a woman Young, strong and tame But soft when she loves me Like a gentle kitten in my hand She makes me feel Like a hell of a man She's one hell of a woman Young, strong and tame But soft when she loves me Like a gentle kitten in my hand She makes me feel Like a hell of a man",
    "Got your lady on the line Got your name on the cover Though your friends are ninety-nine Honey you ain't got no lover And you ain't, you ain't, you ain't got no lover And you ain't, you ain't, you ain't got no other I remember the feeling that I could be free Now I know it could never, never be me 'Cause I'm on fire Got myself on fire Got your joker on the table You've been told from time to time I'll be willin' I'll be able You could read between the lines But you ain't, you ain't, you ain't got no lover Lover, lover And you ain't, you ain't, you ain't got no other I remember the feeling that I could be free Now I know it could never, never be me 'Cause I'm on fire I'm'a, I'm on fire But you ain't, you ain't, you ain't got no lover Lover, lover, lover And you ain't, you ain't, you ain't got no other Other, other, other And you ain't, you ain't, you ain't got no lover Lover, lover I'm on fire, I'm on fire But you ain't, you ain't, you ain't got no lover Lover, lover, lover I'm on fire Now lover, lover, lover, lover I'm on fire Now lover, lover, lover, lover I'm on fire...",
    "Baby hair with a woman's eyes I can feel you watching in the night All alone with me and we're waiting for the sunlight When I feel cold, you warm me And when I feel I can't go on, you come and hold me It's you and me forever Sara smile Won't you smile a while for me, Sara If you feel like leaving, you know you can go But why don't you stay until tomorrow? If you want to be free, you know, all you got to do is say so And when you feel cold, I'll warm you And when you feel you can't go on, I'll come and hold you It's you and me forever Sara smile Won't you smile a while for me, Sara Sara smile Won't you smile a while for me, Sara Smile Won't you smile a while for me, Sara Oh smile a while, won't you laugh Sara Make me feel like a man not keeping me crazy crazy Smile a while",
    "For so long You and me been findin' each other for so long And the feelin' that I feel for you is More than strong, girl Take it from me If you give a little more than you're askin' For your love will turn the key Darlin' mine I would wait forever for those lips of wine Build my world around you, darlin' This love will shine girl, watch it and see If you give a little more than you're askin' for Your love will turn the key I, I, I just wanna be your everything Open up the heaven in your heart and let me be The things you are to me And not some puppet on a string Oh, if I, if I stay here without you darlin' I will die I want you layin' in the love I have to bring I'd do anything to be your everything Darlin' for so long You and me been findin' each other for so long And the feelin' that I feel for you is More than strong girl Take it from me If you give a little more than you're askin' for Your love will turn the key I, I, I just wanna be your everything Open up the heaven in your heart and let me be The things you are to me And not some puppet on a string Oh, if I, if I stay here without you darlin' I will die I want you layin' in the love I have to bring I'd do anything to be your everything My everything I, I, I just wanna be your everything Open up the heaven in your heart and let me be The things you are to me And not some puppet on a string Oh, if I, if I stay here without you darlin' I will die I want you layin' in the love I have to bring I'd do anything to be your everything",
    "Danger in the shape of somethin' wild Stranger dressed in black, she's a hungry child No one knows who she is or what her name is I don't know where she came from or what her game is Hot child in the city Hot child in the city Runnin' wild and lookin' pretty Hot child in the city So young to be loose and on her own Young boys, they all want to take her home She goes downtown, the boys all stop and stare When she goes downtown, she walks like she just don't care, care Hot child in the city Hot child in the city Runnin' wild and lookin' pretty Hot child in the city Come on down to my place, baby We'll talk about love Come on down to my place, woman We'll make love! Hot child in the city (Hot child in the city) She's kinda dangerous (Hot child in the city) Young child (Runnin' wild and lookin' pretty) Young child, runnin' wild (Hot child in the city) Hot child in the city (Hot child in the city) (Hot child in the city) (Hot child in the city) Hot child in the city (Hot child in the city) Hot child in the city (Hot child in the city) Oh!",
    "Looks like it's over, you knew I couldn't stay She's comin' home today We had a good thing, I'll miss your sweet love Why must you look at me that way? It's over Sad eyes, turn the other way I don't want to see you cry Sad eyes, you knew there'd come a day When we would have to say goodbye Try to remember the magic that we shared In time your broken heart will mend I never used you, you knew I really cared I hate to see it have to end But it's over Sad eyes, turn the other way I don't want to see you cry Sad eyes, you knew there'd come a day When we would have to say goodbye Sad eyes, turn the other way (turn the other way) I don't want to see you cry (cry, cry, cry) Sad eyes, you knew there'd come a day (he-e-e-ey) When we would have to say goodbye",
    "I'm lying alone with my head on the phone Thinking of you till it hurts I know you hurt too but what else can we do Tormented and torn apart I wish I could carry your smile and my heart For times when my life seems so low It would make me believe what tomorrow could bring When today doesn't really know, doesn't really know I'm all out of love, I'm so lost without you I know you were right believing for so long I'm all out of love, what am I without you I can't be too late to say that I was so wrong I want you to come back and carry me home Away from this long lonely nights I'm reaching for you, are you feeling it too Does the feeling seem oh so right And what would you say if I called on you now And said that I can't hold on There's no easy way, it gets harder each day Please love me or I'll be gone, I'll be gone I'm all out of love, I'm so lost without you I know you were right believing for so long I'm all out of love, what am I without you I can't be too late to say that I was so wrong Oh, what are you thinking of What are you thinking of What are you thinking of What are you thinking of I'm all out of love, I'm so lost without you I know you were right believing for so long I'm all out of love, what am I without you I can't be too late I know I was so wrong I'm all out of love, I'm so lost without you I know you were right believing for so long I'm all out of love, what am I without you I can't be too late I know I was so wrong I'm all out of love, I'm so lost without you I know you were right believing for so long I'm all out of love, what am I without you I can't be too late to say that I was so wrong",
    "Jessie is a friend Yeah, I know he's been a good friend of mine But lately something's changed, it ain't hard to define Jessie's got himself a girl and I want to make her mine And she's watching him with those eyes And she's lovin' him with that body, I just know it Yeah, and he's holding her in his arms late, late at night You know, I wish that I had Jessie's girl I wish that I had Jessie's girl Why can't I find a woman like that? I'll play along with this charade That doesn't seem to be a reason to change You know I feel so dirty when they start talking cute I wanna tell her that I love her but the point is probably moot 'Cause she's watching him with those eyes And she's lovin' him with that body, I just know it And he's holding her in his arms late, late at night You know, I wish that I had Jessie's girl I wish that I had Jessie's girl Why can't I find a woman like that? Like Jessie's girl I wish that I had Jessie's girl Why can't I find a woman? Why can't I find a woman like that? And I'm lookin' in the mirror all the time Wonderin' what she don't see in me I've been funny, I've been cool with the lines Ain't that the way love's supposed to be Tell me, why can't I find a woman like that? You know, I wish that I had Jessie's girl I wish that I had Jessie's girl I want Jessie's girl Why can't I find a woman like that? Like Jessie's girl I wish that I had Jessie's girl I want, I want Jessie's girl",
    "Sometimes I feel I've got to Run away, I've got to Get away from the pain you drive into the heart of me The love we share Seems to go nowhere And I've lost my light For I toss and turn, I can't sleep at night Once I ran to you (I ran) Now I'll run from you This tainted love you've given I give you all a boy could give you Take my tears and that's not nearly all Tainted love (oh) Tainted love Now I know I've got to Run away, I've got to Get away, you don't really want any more from me To make things right You need someone to hold you tight And you think love is to pray But I'm sorry, I don't pray that way Once I ran to you (I ran) Now I'll run from you This tainted love you've given I give you all a boy could give you Take my tears and that's not nearly all Tainted love (oh) Tainted love Don't touch me, please I cannot stand the way you tease I love you though you hurt me so Now I'm gonna pack my things and go Tainted love (oh) Tainted love (oh) Tainted love (oh) Tainted love (oh) Touch me, baby, tainted love Touch me, baby, tainted love Tainted love (oh) Tainted love (oh) Tainted love Tainted love",
    "Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas Everybody's looking for something Some of them want to use you Some of them want to get used by you Some of them want to abuse you Some of them want to be abused Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas Everybody's looking for something Hold your head up, keep your head up, movin' on Hold your head up, movin' on, keep your head up, movin' on Hold your head up, movin' on, keep your head up, movin' on Hold your head up, movin' on, keep your head up Some of them want to use you Some of them want to get used by you Some of them want to abuse you Some of them want to be abused Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas Everybody's looking for something Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas Everybody's looking for something Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas Everybody's looking for something Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas Everybody's looking for something Sweet dreams are made of this Who am I to disagree I travel the world and the seven seas",
    "Something in the way you love me won't let me be I don't want to be your prisoner So baby, won't you set me free? Stop playing with my heart Finish what you start When you make my love come down If you want me, let me know Baby, let it show Honey, don't you fool around Just try to understand I've given all I can 'Cause you got the best of me Borderline Feels like I'm going to lose my mind You just keep on pushing my love over the borderline Borderline Feels like I'm going to lose my mind You just keep on pushing my love over the borderline Something in your eyes is makin' such a fool of me When you hold me in your arms You love me till I just can't see But then you let me down When I look around Baby, you just can't be found Stop driving me away I just wanna stay There's something I just got to say Just try to understand I've given all I can (Motherfucker) 'Cause you got the best of me Borderline Feels like I'm going to lose my mind You just keep on pushing my love over the borderline Borderline Feels like I'm going to lose my mind You just keep on pushing my love over the borderline Keep pushing me, keep pushing me Keep pushing my love, c'mon baby, c'mon darling, yeah Keep on pushing my love over the borderline, borderline",
    "Every day I hear a different story People saying that you're no good for me Saw your lover with another and she's makin' a fool of you, oh If you love me baby, you'd deny it But you laugh and tell me I should try it Tell me I'm a baby, and I don't understand But you know that I'll forgive you Just this once, twice, forever 'Cause baby, you could drag me to hell and back Just as long as we're together And you do I don't want your freedom I don't want to play around I don't want nobody, baby Part time love just brings me down I don't need your freedom Girl, all I want right now is you Do, do, do Whoa, oh, yeah Do-do-do-do-do-do Do-do-do-do-do-do Like a prisoner who has his own key But I can't escape until you love me I just go from day to day knowing all about the other boys You take my hand and tell me I'm a fool to give you all that I do I bet you someday, baby, someone says the same to you But you know that I'll forgive you Just this once, twice, forever 'Cause baby, you could drag me to hell and back Just as long as we're together And you do, oh I don't want your freedom I don't want to play around I don't want nobody, baby Part time love just brings me down I don't want your freedom Girl, all I want right now is you Do, do, do Can't you see you're hurtin' me? Whoa, oh, yeah Do-do-do-do-do-do Do-do-do-do-do-do You hurt me baby, hurt me baby Oh Do-do-do, do, do, do, do Do-do-do, do, do, do, do Do-do-do, do, do, do, do Do-do-do, do, do, do, do You hurt me baby, hurt me baby But you know that I'll forgive you Just this once, twice, forever 'Cause baby, you could drag me to hell and back Just as long as we're together And you do, oh I don't want your freedom I don't need to play around I don't want nobody, baby Part time love just brings me down I don't want your freedom I don't want to play around I don't want nobody baby Part time love just brings me down I don't want your (I don't want your) I don't want your (I don't want your) I don't want your (I don't want your) I don't want your part time love Oh",
    "I can't wait I love you I can't wait I can't wait Sometime she talks to him Sometimes when she's only dreaming Then when she wakes up Somehow, well, she still believes him Yes, I know you Know we've been out of touch Yes, I know you To be continued, it's too much Well, I can't wait, I can't wait Well, if I'm wasting my time, if you make up your mind Well, I can't wait, I can't wait And I've got to know how if I'll see you again Well, I can't wait, I can't wait Well, yes, I know you Know we've been out of touch I can't wait, I can't wait What can I do when I'm crazy for you? She wonders how many more hours Her heart will feel broken In secret, she says she needs to see him But no words are spoken Well, she dances around in a circle Well, she's got that feeling now Blame it on something at first sight Put the blame on me if you want to To be continued I can't wait, I can't wait Am I wasting my time while you make up your mind, now babe? Well, I can't wait I can't wait Now I got to know when I can see you again Well, I can't wait, I can't wait What can I do when I'm crazy for you, baby? I can't wait, I can't wait Am I wasting my time? Make up your mind, baby I love you I love you How will we feel 20 years from now? How will we feel 20 years from now? How will we feel 20 years from now? How will we feel 20 years from now? How will we feel 20 years from now? How will we feel 20 years from now? Well, I can't wait, I can't wait If I'm wasting my time while you make up your mind, baby I can't wait, I can't wait And I got to know when I can see you again Well, I can't wait, I can't wait No, no time will, make up your mind, baby I can't wait, I can't wait I love, love you, I love, love you",
    "Something in the moonlight catches my eye The shadow of a lover goes dancing by Looking for a little bit of love to grow, so Give me love, give me heart and soul You never let me cross to the other side now I'm tied to the hope that you will somehow Hard on the heels of something more But I lost your love, heart and soul The tear in my heart as you walk on by I feel so low and your head is high Everything you do convinces me more Please, give me love, give me heart and soul Looking to the day when I saw your face I wasn't in the running, I wasn't in the race You move in a way that I've known before Now I want your love, heart and soul Leaving you ain't easy now Loving you's the harder part You never want me for myself I've needed you right from the very start Oh won't you even try to Give a little bit of heart and soul Give a little bit of love to grow Give a little bit of heart and soul Don't you make me beg for more Give a sign, I need to know A little bit of heart and soul Walking on the water, walking on the air That was the heart of the love we shared Do you keep secret left untold Can't give love, heart or soul I used to have a lover with a Midas touch I turned to gold but he turned to dust Left me for another, I turned to stone Now I want your love, heart and soul Living in a fantasy There's never any room to breathe Hoping every waking hour You gonna turn around and say that we can start Oh won't you even try to Give a little bit of heart and soul Give a little bit of love to grow Give a little bit of heart and soul Don't you make me beg for more Give a sign, I need to know A little bit of heart and soul Somehow, I lost my way Looking to see something in your eyes But love will never compromise Now this is the politics of life, yeah Give a little bit of heart and soul Give a little bit of love to grow Give a little bit of heart and soul Don't you make me beg for more Give a sign, I need to know A little bit, little bit Give a little bit of heart and soul Give a little bit of love to grow Give a little bit of heart and soul Don't you make me beg for more Give a sign, I need to know A little bit of heart now want your soul?",
    "Girl, the only thing that matters in my life Is that I'm down for you and treat you right You've got no call to treat me cold as ice Oh oh, girl As long as I've been givin' my love to you You should be givin' me your love, too But you just keep on actin' just like a fool You know it ain't cool Uh don't be cruel 'Cause I would never be that cruel to you Uh no, oh oh, no Uh don't be cruel Uh girl, you need to change your attitude Uh no, oh Uh don't be cruel Hey yo' Kimmy what's up with this attitude I thought I was bein' real good to you I treat you sweet, take you out at night But you never say thanks girl that ain't right I bought you diamonds even gave you pearls I took you for a cruise all around the world I treat you high post but you play me close If I want to drink up you won't even toast Girl, I work so hard for you from 9 to 5 So you could have the finer things in life Since you're the kind that's never satisfied Oh oh, girl As long as I've been givin' my heart to you You should be givin' me your heart, too But you just keep on actin' just like a fool You know it ain't cool Uh don't be cruel 'Cause I would never be that cruel to you Uh no, oh oh, no Uh don't be cruel Uh girl, you need to change your attitude Uh no, oh Uh don't be cruel From fancy cars to diamond rings I've just about given you everything There's really not much I won't do for you I bought you 12 yellow roses and candy, too I like be up front and never play the back But the way you treat me girl is like a heart attack I'm real troop trooper and with all the times That you were by my side diggin' dollars outta crock Oh oh, girl As long as I've been givin' my love to you You should be givin' me your love, too But you just keep on actin' just like a fool You know it ain't cool It just ain't cool! Ow! Don't be cruel Uh, uh, uh Uh, uh, uh We used to hang out tough just kickin' around We discovered a love that had never been found You gave me your heart I gave you my mind But a true love affair we could never find Although I want you bad I could let you go 'Cause there's a lot of girls out there that won't say no But due to fact that I want you, Jackie I want you more than human eyes can see But you had to start givin', tryin' to make a killin' Thought about the dollars I make and you were willin' To be with me Bobby B., but with a bad attitude I can't compete Now you know my name, now I know your game You want to be with me you got to be the same way That another girl would be and if you want to be with me Uh don't be cruel 'Cause I would never be that cruel to you Uh no, oh oh, no Uh don't be cruel Uh girl, you need to change your attitude Uh no, oh Uh don't be cruel 'Cause I would never be that cruel to you Uh no, oh oh, no Uh don't be cruel Uh girl, you need to change your attitude Uh no, oh We used to hang out tough just kickin' around We discovered a love that had never been found You gave me your heart I gave you my mind But a true love affair we could never find Although I want you bad I could let you go 'Cause there's a lot of girls out there that won't say no But due to fact that I want you, Jackie I want you more than human eyes can see But you had to start givin', tryin' to make a killin' Thought about the dollars I make and you were willin' To be with me Bobby B., but with a bad attitude I can't compete Now you know my name, now I know your game You want to be with me you got to be the same way That another girl would be and if you want to be with me Uh don't be cruel 'Cause I would never be that cruel to you Uh no, oh oh, no Uh don't be cruel Uh girl, you need to change your attitude Uh no, oh Uh don't be cruel",
    "Whenever I'm alone with you You make me feel like I am home again Whenever I'm alone with you You make me feel like I am whole again Whenever I'm alone with you You make me feel like I am young again Whenever I'm alone with you You make me feel like I am fun again However far away I will always love you However long I stay I will always love you Whatever words I say I will always love you I will always love you Whenever I'm alone with you You make me feel like I am free again Whenever I'm alone with you You make me feel like I am clean again However far away I will always love you However long I stay I will always love you Whatever words I say I will always love you I will always love you",
    "I know there's pain (I know there's pain) Why do you lock yourself up in these chains? (These chains) No one can change your life except for you Don't ever let anyone step all over you Just open your heart and your mind (Mmm) Is it really fair to feel, this way inside? (Woah) Some day somebody's gonna make you want to turn around and say goodbye Until then, baby, are you going to let 'em hold you down and make you cry? Don't you know? Don't you know, things can change Things'll go your way If you Hold On for one more day Can you Hold On for one more day? Things'll go your way Hold On for one more day You could sustain (You could sustain) Hm, or are you comfortable with the pain? You've got no one to blame for your unhappiness (No, baby) You got yourself into your own mess (Ooh) Lettin' your worries pass you by (Lettin' your worries pass you by) Baby, don't you think it's worth your time To change your mind? (No, no) Some day somebody's gonna make you want to turn around and say goodbye Until then, baby, are you going to let 'em hold you down and make you cry? Don't you know? Don't you know, things can change Things'll go your way If you Hold On for one more day Can you Hold On for one more day? Things'll go your way (Oh, things'll go your way) Hold On for one more day I know that there is pain, but you Hold On for one more day, and ya Break free from the chains Yeah I know that there is pain, but you Hold On for one more day, and ya Break free, break from the chains Some day somebody's gonna make you want to turn around and say goodbye (and say goodbye) Until then, baby, are you going to let 'em hold you down and make you cry? Don't you know? Don't you know, things can change (Know) Things'll go your way If you Hold On for one more day, yeah If you Hold On Don't you know, things could change Things could go your way If you Hold On for one more day Can you Hold On Can you Hold On Mmm, can you Hold On, baby Won't you tell me now Hold On for one more day, 'cause It's gonna go your way Don't you know, things could change Things could go your way If you Hold On for one more day, yeah Can't you change it this time Make up your mind Hold On, Hold On Baby Hold On",
    "Babyyy Babyyy Hi it's me once again My name is Daisy Dee I wanna show you how to party me and MC B The bass is pumpin' into your body Don't think about it C'mon get started It doesn't matter if you're infirm Dance to the rhythm is your concern We get up, get crazy, healling ....... and jack you're chilling Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Dance to the rhythm, shake your body Gimme your attention if you wanna party Yeah, c'mon let's do it Now it's the time to get into it Black, the color of the skin, no deny Rap is the way to express all the rimes Wherever you're in trouble or ever you're down I'll keep your body pumpin' to the movin' sound Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Stop moving the mic I'm doing my case I'm comin' on strong ........ So free your mind and go into it Take your time, and let's hell do it Don't forget the name is Daisy Operation's in the house to make you crazy Direction? I won't tell you Actually the way to tell you what to do Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Dance to the rhythm for me to make you crazy Don't think that I can make you leavin' Dance to the rhythm for me to make you crazy Don't think that I can make you leavin'",
    "Hey baby I'm sorry I never meant to hurt you Please don't go Slowly my eyes began to see That I need you near right with me at all times, yeah My feelings are so deep for you That I won't let go, oooh no, of you I can't let this love (slip away) And I can't turn it around (you will see) And only then Please don't go away from me (I'll be there for you) When you call my name (call my name) I'll reach out my hand to you (And I'll welcome you) To my heart (my future my heart) Please don't go away from me We can work it out Whatever it may be girl Please don't go (no, no, no, no, no, please don't go) You know, It's not often that we see each other, That's why I cherish every moment that we spent each other. When you're away, temptation may come Above of when I get caught up (by you) (3x), I let go Thinking about you each day I feel (I love you and I'll never let go) Your love makes my head spin Round and round in a daze, yeah oh baby I can't let this love (slip away) And I can't turn it around (you will see) And only then Please don't go away from me (I'll be there for you) When you call my name (call my name) I'll reach out my hand to you (And I'll welcome you) To my heart (my future my heart) Please don't go away from me We can work it out Whatever it may be girl Please don't go (no, no, no, no, no, please don't go) Please don't go away I'll be there, I'll be right there I'll be there, I'll be right there I'll be there, I'll be right there I'll be there, I'll be right there I'll be there, I'll be right there I'll be there, I'll be right there I'll be there, I'll be right there",
    "Come on baby kick those Dazzeys Look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Everybody look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Say look at them girls with the Dazzey Duks on They really got it going on Looking all nice and sweet Showing leg and a little butt cheek I do like it a lot When you're showing some of what you got why y'all girls be looking real cute The way why y'all kicking those Dazzey Duks Come on baby kick those Dazzeys Look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Everybody look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Hey, you, in the black boots Knee high with the Dazzey Duks Looking real fly while in cahoots In the park where I can meet you I'm Snow, here to relate to you What started the trend of them Dazzey Duks Ever since the summer of the 90s Girls had a deal with the future behind See many terms they use to express We're talking about the butt cheeks not the breasts Baby got back, junk in the trunk She got a six pack or a hell of a rump Yo, need some fries with that shake See many terms they use to relate I love the girls with the big old booties Right up front they call em Dukies Pretty brown cheeks hanging from the jeans Even city girls can be country things So if you get it got it good to dip to dug out Dem damn Dazzey Duks are turning out Sing! Come on baby kick those Dazzeys Look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Everybody look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Say look at them girls with the Dazzey Duks on They really got it going on Yeah they be kicking it sweet While all the brothers show their freak And check out the way they step Make a brother want to think about sex Especially when they have nice legs And they're tan to the top all the way But it don't matter what kinda legs you like Still be kicking if those legs are lily white So it makes no difference to you As long as you're kicking them Dazzey Duks Come on baby kick those Dazzeys Look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Everybody look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Hold it now, wait a minute, yo! Bring that funky track back Bring that funky track back Bring that funky track back Bring that funky track back Oh, bring it baby, bring it baby, bring it baby Kick those Dazzeys Bring that funky track back Bring it baby, bring it baby Oh let me see you rock baby Bring it baby, bring it baby Bring that funky track back Oh, bring it baby, bring it baby, bring it baby Kick those Dazzeys Bring that funky track back Bring it baby, bring it baby Oh let me see you rock baby Bring it baby, bring it baby Kick those Dazzeys Bring that funky track back Bring it baby, bring it baby Oh let me see you rock baby LA Snow like the girls with the Dazzey Duks Creo D like the girls with the Dazzey Duks Taylor Boy like them girls in the Dazzey Duks Tony Mercedes like them girls in the Dazzey Duks New York, Bel Air, Miami, Alabama, Tennessee, New Jersey, Boston, Ausgusta Atlanta like them girls in the Dazzey Duks All of Texas like them girls in the Dazzey Duks LA like them girls in the Dazzey Duks Come on baby kick those Dazzeys Look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Everybody look at them girls with the Dazzey Duks on I want you to look at them girls with the Dazzey Duks on Oh kick those Dazzeys She should wear em Kick those Dazzeys baby She should wear em wear em Kick those Dazzeys baby Hold it now, wait a minute",
    "I swear by the moon and the stars in the skies And I swear like the shadow that's by your side I see the questions in your eyes I know what's weighing on your mind You can be sure I know my part 'Cause I'll stand beside you through the years You'll only cry those happy tears And though I make mistakes I'll never break your heart And I swear by the moon and the stars in the skies I'll be there I swear like a shadow that's by your side I'll be there For better or worse till death do us part I'll love you with every beat of my heart And I swear Ooh I'll give you everything I can I'll build your dreams with these two hands We'll hang some memories on the walls And when (and when) Just the two of us are there You won't have to ask if I still care 'Cause as the time turns the page my love won't age at all And I swear (I swear) By the moon and the stars in the skies I'll be there (I'll be there) I swear (and I swear) Like the shadow that's by your side I'll be there (I'll be there) For better or worse till death do us part I'll love you with every beat of my heart (beat of my heart) And I swear (oh yeah) I swear (I swear) By the moon and stars in the sky I'll be there (I'll be there) (I swear) I swear like the shadow that's by your side I'll be there (I'll be there) For better or worse (better or worse) Till death do us part (oh no) (I'll love you with every) every single beat of my heart (I swear) I swear (I swear) Oh, I swear",
    "Oh, once upon a midnight eerie I woke with something in my head I couldn't escape the memory Of a phone call and of what you said Like a game show contestant with a parting gift I could not believe my eyes When I saw through the thoughts of a trusted friend Who needs to humor me and tell me lies, yeah humor me and tell me lies And I'll lie too and say I don't mind And as we seek so shall we find And when you're feeling open I'll still be here But not without a certain degree of fear Well, what will be with you and me I still can see things hopefully But you Why you wanna give me a run-around? Is it a sure-fire way to speed things up? When all it does is slow me down Shake me and my confidence About a great many things But I've been there I can see it cower Like a nervous magician waiting in the wings Of a bad play where the heroes are right And nobody thinks or expects too much And Hollywood's calling for the movie rights Singing Hey babe, let's keep in touch, hey baby, let's keep in touch But I want more than a touch I want you to reach me And show me all the things no one else can see So what you feel becomes mine as well And soon if we're lucky we'd be unable to tell What's yours and mine, the fishing's fine And it doesn't have to rhyme, so don't you feed me a line But you Why you wanna give me a run-around? Is it a sure-fire way to speed things up? When all it does is slow me down Tra la la la la bombardier, this is the pilot speaking I've got some news for you It seems my ship still stands no matter what you drop And there ain't a whole lot that you can do Oh sure the banner may be torn and the wind's gotten colder Perhaps I've grown a little cynical But I know no matter what the waitress brings I shall drink it and always be full, yeah I will drink it and always be full Oh I like coffee and I like tea I'd like to be able to enter a final plea I still got this dream that you just can't shake I love you to the point you can no longer take Well all right okay, so be that way I hope and pray that there's something left to say But you Why you wanna give me a run-around? Is it a sure-fire way to speed things up When all it does is slow me down? Oh you Why you wanna give me a run-around? Is it a sure-fire way to speed things up When all it does is slow me down?",
    "I didn't ask They shouldn't have told me At first I'd laugh, but now It's sinking in fast Whatever they've sold me Well baby, I don't want to take advice from fools I'll just figure everything is cool Until I hear it from you It gets hard The memory's faded Who gets what they say It's likely they're just jealous and jaded Well maybe I don't want to take advice from fools I'll just figure everything is cool Until I hear it from you Until I hear it from you I can't let it get me off Or break up my train of thought As far as I know, nothing's wrong Until I hear it from you Still thinking about not living without it Outside looking in 'Til we're talking about it, not stepping around it Maybe I don't want to take advice from fools I'll just figure everything is cool Until I hear it from you (hear it from you) Until I hear it from you (hear it from you) Until I hear it from you (hear it from you) Until I hear it from you, oh Until I hear it from you. oh no Until I hear it from you (won't take advice from fools) Ooh (I'll figure everything is cool) Until I hear it from you (won't take advice from fools) Oh (I'll figure everything is cool) Until I hear it from you",
    "You took your coat off, stood in the rain You're always crazy like that And I watched from my window Always felt I was outside looking in on you You're always the mysterious one with Dark eyes and careless hair You were fashionably sensitive But too cool to care You stood in my doorway, with nothing to say Besides some comment on the weather Well, in case you failed to notice In case you failed to see This is my heart bleeding before you This is me down on my knees And these foolish games are tearing me apart And your thoughtless words are breaking my heart You're breaking my heart You were always brilliant in the morning Smoking your cigarettes and talking over coffee Your philosophies on art, Baroque moved you You loved Mozart and you'd speak of your loved ones As I clumsily strummed my guitar You'd teach me of honest things Things that were daring, things that were clean Things that knew what an honest dollar did mean I hid my soiled hands behind my back Somewhere along the line, I must have gone off track with you Excuse me Think I've mistaken you for somebody else Somebody who gave a damn Somebody more like myself And these foolish games are tearing me You're tearing me, you're tearing me apart And your thoughtless words are breaking my heart You're breaking my heart And you took your coat off, stood in the rain You're always crazy like that",
    "I'll be your dream, I'll be your wish, I'll be your fantasy I'll be your hope, I'll be your love, be everything that you need I love you more with every breath truly, madly, deeply do I will be strong, I will be faithful 'cause I'm counting on A new beginnin' A reason for livin' A deeper meaning, yeah I wanna stand with you on a mountain I wanna bathe with you in the sea I wanna lay like this forever Until the sky falls down on me And when the stars are shining brightly in the velvet sky I'll make a wish, send it to heaven and make you want to cry The tears of joy for all the pleasure and the certainty That we're surrounded by the comfort and protection Of the highest powers In lonely hours The tears devour you An' I wanna stand with you on a mountain I wanna bathe with you in the sea I wanna lay like this forever Until the sky falls down on me Oh, can you see it, baby? You don't have to close your eyes It's standin' right before you All that you need will surely come Ooh, yeah I'll be your dream, I'll be your wish, I'll be your fantasy I'll be your hope, I'll be your love, be everything that you need I'll love you more with every breath truly, madly, deeply do I wanna stand with you on a mountain I wanna bathe with you in the sea I wanna lay like this forever Until the sky falls down on me I wanna stand with you on a mountain I wanna bathe with you in the sea I want to live like this forever Until the sky falls down on me", 
    "Uh huh, uh huh, Fabolous, 702 Uh huh, uh huh, uh huh Uh huh, uh huh Now of these other chicks don't even come close to you Anythin' less than a dime, ain't negotiable There holla Fabolous when I swing the Testarossa through Maybe cause I'm close to Clue, I be in posters too I rock so much ice, you need shades for a closer view I beep the horn, that's the most I do Like what this player supposed to do Fuck it, pack your bags, let's spend the weekend in Costa boo You feelin' that? See he's my property and any girl that touch I might just call your bluff 'Cause I don't give a fuck Who are you to call my cell Oh I'm a wish you well Cause any girl that tried has fell Where my girls at From the front to back Well is you feelin' that Well put one hand up, can you repeat that Tryin' to take my man, see I don't need that So don't play yourself Where my girls at From the front to back Well is you feelin' that Well put one hand up, can you repeat that Tryin' to take my man, see I don't need that Hey, hey, hey, hey Don't you violate me Cause I'm a make ya hate me If you decide to mess with mine Shrunk you down to size Make ya realize You done messed up this time You must learn the rules Don't come close to Anything that belongs to me He is my, my property So you must leave Uh huh, uh huh, Where my girls at? Fabolous, F-A-be -O-L-O-you-S Uh, uh, uh Don't front ma, who kept you rockin' all the stones? Hit the mall, you can pick and roll like Stockton and Malone Must of dropped bread on my skew, girls be flockin' to my home When I got a case, quick tryin' to get Cochran on the phone Chicks don't want you in karats, that make your pinkie finger glow They want to argue and are fightin' on the Jerry Springer show Yours friends want Fabolous, there's more fishes in the sea Schemin' since I put the deep dishes on the V Just to let you know, it's nothin' when you ask for green Took you to Cristal, for them cats and cream I'm the one that had you rockin Icebergs to affairs Icebergs for your gear, Icebergs in your hair I don't know how them dirty birds got the cell digits When I come home you pat me down like jail visits All I want to do is stay close to you Fabolous, Desert Storm, 702 Ya heard You must learn the rules Don't come close to Anything that belongs to me He is my, my property So you must leave",
    "I can feel the magic floating in the air Being with you get's me that way I watch the sunlight dance across your face And I've never been this swept away All my thoughts just seem to settle on the breeze When I'm lying wrapped up in your arms The whole world just fades away The only thing I hear Is the beating of your heart 'Cause I can feel you breathe It's washing over me And suddenly I'm melting into you There's nothing left to prove Baby, all we need is just to be Caught up in the touch Slow and steady rush Baby, isn't that the way that love's suppose to be I can feel you breathe, just breathe In a way I know my heart is waking up As all the walls come tumbling down Closer than I've ever felt before And I know, and you know There's no need for words right now I can feel you breathe Washing over me And suddenly I'm melting into you There's nothing left to prove Baby, all we need is just to be Caught up in the touch Slow and steady rush Baby, isn't that the way that love's suppose to be I can feel you breathe, just breathe Caught up in the touch Slow and steady rush Baby, isn't that the way that love's suppose to be I can feel you breathe, just breathe I can feel the magic floating in the air Being with you gets me that way",
    "Desperate for changing Starving for truth I'm closer to where I started I'm chasing after you I'm falling even more in love with you Letting go of all I've held on to I'm standing here until you make me move I'm hanging by a moment here with you Forgetting all I'm lacking Completely incomplete I'll take your invitation You take all of me now I'm falling even more in love with you Letting go of all I've held on to I'm standing here until you make me move I'm hanging by a moment here with you I'm living for the only thing I know I'm running and not quite sure where to go And I don't know what I'm diving into Just hanging by a moment here with you There's nothing else to lose There's nothing else to find There's nothing in the world That can change my mind There is nothing else There is nothing else There is nothing else Desperate for changing Starving for truth I'm closer to where I started I'm chasing after you I'm falling even more in love with you Letting go of all I've held on to I'm standing here until you make me move I'm hanging by a moment here with you I'm living for the only thing I know I'm running and not quite sure where to go And I don't know what I'm diving into Just hanging by a moment here with you Just hanging by a moment Hanging by a moment Hanging by a moment Hanging by a moment here with you",
    "Let me be your hero Would you dance, if I asked you to dance? Would you run, and never look back? Would you cry, if you saw me cryin'? And would you save my soul tonight? Would you tremble, if I touched your lips? Would you laugh? Oh, please, tell me this Now, would you die for the one you loved? Hold me in your arms tonight I can be your hero, baby I can kiss away the pain I will stand by you forever You can take my very breath away Would you swear that you'll always be mine? Or would you lie? Would you run and hide? Am I in too deep? Have I lost my mind? I don't care, you're here tonight I can be your hero, baby I can kiss away the pain I will stand by you forever You can take my breath away Oh, I just wanna hold you I just want to hold you, oh, yeah Am I in too deep? Have I lost my mind? Well, I don't care, you're here tonight I can be your hero, baby I can kiss away the pain, oh, yeah I will stand by you forever You can take my breath away And I can be your hero I can kiss away the pain And I will stand by you forever You can take my breath away You can take my breath away I can be your hero",
    "Don't look at me Every day is so wonderful Then suddenly it's hard to breathe Now and then I get insecure From all the pain I'm so ashamed I am beautiful no matter what they say Words can't bring me down I am beautiful in every single way Yes, words can't bring me down, oh no So don't you bring me down today To all your friends you're delirious So consumed in all your doom Trying hard to fill the emptiness The pieces gone, left the puzzle undone Is that the way it is? You are beautiful no matter what they say Words can't bring you down, oh no You are beautiful in every single way Yes, words can't bring you down, oh no So don't you bring me down today No matter what we do (No matter what we do) No matter what we say (No matter what we say) We're the song inside the tune (Yeah) Full of beautiful mistakes And everywhere we go (And everywhere we go) The sun will always shine (The sun will always, always shine) And tomorrow we might wake on the other side 'Cause we are beautiful no matter what they say Yes, words won't bring us down, no, no We are beautiful in every single way Yes, words can't bring us down, oh, no So don't you bring me down today Ooh-oh-oh, yeah Don't you bring me down today Yeah, ooh Don't you bring me down Ooh, today", 
    "Peace up, A-Town down Yeah! Okay (Usher, Usher, Usher) Lil' Jon Yeah, yeah, yeah, yeah, yeah, yeah Yeah, yeah, yeah, yeah, yeah, yeah Up in the club with my homies, trying to get a lil' V-I Keep it down on the low key You should know how it feels (hey) I seen shorty, she was checking up on me From the game she was spitting in my ear You would think that she know me I decided to chill (okay!) Conversation got heavy She had me feeling like she's ready to blow (watch out!) Oh! (watch out) She's saying, come get me So I got up and followed her to the floor She said, baby let's go When I told her (let's go!) I said Yeah! Shorty got down low said, come and get me (Yeah!) Yeah! I got so caught up, I forgot she told me (Yeah!) Yeah! Her and my girl, they used to be the best of homies (Yeah!) Yeah! Next thing I knew, she was all up on me screaming Yeah (yeah!) yeah (yeah!) yeah, yeah Yeah (yeah!) yeah (yeah!) yeah, yeah She's all up in my head now, got me thinking That it might be a good idea to take her with me 'Cause she's ready to leave, ready to leave now (let's go!) And that I gotta keep it real now 'Cause on a one to ten, she's a certified twenty But that just ain't me (hey!) 'Cause I don't know, if I take that chance Just where's it gonna lead But what I do know is the way she dance Make shorty alright with me The way she get low I'm like, yeah, just work that out for me She asks for one more dance, and I'm like, yeah How the hell am I supposed to leave (Let's go, bring the beat back!) And I say Yeah! Shorty got down low said, come and get me (Yeah!) Yeah! I got so caught up, I forgot she told me (Yeah!) Yeah! Her and my girl, they used to be the best of homies (Yeah!) Yeah! Next thing I knew, she was all up on me screaming Yeah (yeah!) yeah (yeah!) yeah, yeah! Yeah (yeah!) yeah (yeah!) yeah, yeah! Watch out, my outfit's ridiculous In the club looking so conspicuous And roar, these women all on the prowl If you hold the head steady, I'ma milk the cow (yeah!) And forget about game, I'ma spit the truth (what?) I won't stop 'til I get 'em in they birthday suit (yeah!) So gimme the rhythm and it'll be off with their clothes Then bend over to the front and touch your toes I left the Jag and I took the Rolls If they ain't cutting, then I put 'em on foot patrol (let's go!) How you like me now? When my pinky's valued over three hundred thousand Let's drink, you the one to please (yeah!) Ludacris fill cups like double D's (yeah!) Me and Ursh' once more, and we leaves 'em dead We want a lady in the street but a freak in the bed that say Yeah! Shorty got down low said, come and get me (Yeah!) Yeah! I got so caught up, I forgot she told me (Yeah!) Yeah! Her and my girl, they used to be the best of homies (Yeah!) Yeah! Next thing I knew, she was all up on me screaming Yeah (yeah!) yeah (yeah!) yeah, yeah! Yeah (yeah!) yeah (yeah!) yeah, yeah! Take that and rewind it back Lil Jon got the beat that make your booty go Take that, rewind it back Ursher got the voice make your booty go Take that, rewind it back Ludacris got the flow to make your booty go Take that, rewind it back Lil Jon got the beat that make your booty go",
    "What day is it And in what month? This clock never seemed so alive I can't keep up, and I can't back down I've been losing so much time 'Cause it's you and me And all of the people with nothing to do Nothing to lose And it's you and me And all of the people And I don't know why I can't keep my eyes off of you What are the things that I want to say Just aren't coming out right? I'm tripping on words You got my head spinning I don't know where to go from here 'Cause it's you and me And all of the people with nothing to do Nothing to prove And it's you and me And all of the people And I don't know why I can't keep my eyes off of you There's something about you now That I can't quite figure out Everything she does is beautiful Everything she does is right 'Cause it's you and me And all of the people with nothing to do Nothing to lose And it's you and me and all of the people And I don't know why I can't keep my eyes off of you and me And all of the people with nothing to do Nothing to prove And it's you and me and all of the people And I don't know why I can't keep my eyes off of you What day is it? And in what month? This clock never seemed so alive",
    "I can take the rain on the roof of this empty house That don't bother me I can take a few tears now and then, and just let 'em out I'm not afraid to cry every once in a while even though Going on with you gone still upsets me There are days every now and again I pretend I'm okay But that's not what gets me What hurts the most Was being so close And having so much to say And watching you walk away And never knowing What could've been And not seeing that love in you Is what I was trying to do It's hard to deal with the pain of losing you everywhere I go But I'm doing it It's hard to force that smile when I see our old friends And I'm alone Still harder getting up, getting dressed, living with this regret But I know if I could do it over I would trade, give away all the words that I saved in my heart That I left unspoken What hurts the most Is being so close And having so much to say (much to say) And watching you walk away And never knowing What could've been And not seeing that love in you Is what I was trying to do Oh Hey, yeah What hurts the most Was being so close And having so much to say (much to say) And watching you walk away And never knowing What could've been And not seeing that love in you Is what I was trying to do Not seeing that love in you That's what I was trying to do Ooh, ooh, ooh, ooh, ooh, ooh",
    "Right now, he's probably slow dancin' with a bleached-blond tramp and she's probably gettin' frisky Right now, he's probably buyin' her some fruity little drink 'cause she can't shoot whiskey Right now, he's probably up behind her with a pool-stick showin' her how to shoot a combo And he don't know I dug my key into the side of his pretty little souped-up four-wheel drive Carved my name into his leather seats I took a Louisville slugger to both head lights I slashed a hole in all four tires Maybe next time he'll think before he cheats Right now, she's probably up singing some white-trash version of Shania karaoke Right now, she's probably sayin' I'm drunk And he's a-thinkin' that he's gonna get lucky Right now, he's probably dabbin' on three dollars worth of that bathroom Polo Oh, and he don't know That I dug my key into the side of his pretty little souped-up four-wheel drive Carved my name into his leather seats I took a Louisville slugger to both head lights I slashed a hole in all four tires Maybe next time he'll think before he cheats I might have saved a little trouble for the next girl A-'cause the next time that he cheats Oh, you know it won't be on me No, not on me 'Cause I dug my key into the side of his pretty little souped-up four-wheel drive Carved my name into his leather seats I took a Louisville slugger to both head lights I slashed a hole in all four tires Maybe next time he'll think before he cheats Oh, maybe next time he'll think before he cheats Oh, before he cheats Oooh",
    "Closed off from love, I didn't need the pain Once or twice was enough and it was all in vain Time starts to pass, before you know it, you're frozen, ooh But something happened for the very first time with you My heart melts into the ground, found something true And everyone's looking 'round, thinking I'm going crazy, oh But I don't care what they say I'm in love with you They try to pull me away, but they don't know the truth My heart's crippled by the vein that I keep on closing You cut me open and I Keep bleeding, keep, keep bleeding love I keep bleeding, I keep, keep bleeding love Keep bleeding, keep, keep bleeding love You cut me open Oh yeah Trying hard not to hear, but they talk so loud Their piercing sounds fill my ears, try to fill me with doubt Yet I know that their goal is to keep me from falling, hey, oh But nothing's greater than the rush that comes with your embrace And in this world of loneliness, I see your face Yet everyone around me thinks that I'm going crazy Maybe, maybe But I don't care what they say I'm in love with you They try to pull me away, but they don't know the truth My heart's crippled by the vein that I keep on closing You cut me open and I Keep bleeding, keep, keep bleeding love I keep bleeding, I keep, keep bleeding love Keep bleeding, keep, keep bleeding love You cut me open And it's draining all of me Though they find it hard to believe I'll be wearing these scars for everyone to see I don't care what they say I'm in love with you They try to pull me away, but they don't know the truth My heart's crippled by the vein that I keep on closing Oh, you cut me open and I Keep bleeding, keep, keep bleeding love I keep bleeding, I keep, keep bleeding love Keep bleeding, keep, keep bleeding love Oh, you cut me open and I Keep bleeding, keep, keep bleeding love I keep bleeding, I keep, keep bleeding love Keep bleeding, keep, keep bleeding love I keep Oh, you cut me open and I Keep bleeding, keep, keep bleeding love",
    "I've been roaming around, Always looking down and all I see Painted faces fill the places I can't reach You know that I could use somebody You know that I could use somebody Someone like you And all you know and how you speak Countless lovers Undercover of the street You know that I could use somebody You know that I could use somebody Someone like you Off in the night while you live it up, I'm off to sleep Waging wars to shake the poet and the beat I hope it's gonna make you notice I hope it's gonna make you notice Someone like me Someone like me Someone like me, somebody Go and let it out Someone like you, somebody Someone like you, somebody Someone like you, somebody I've been roaming around, Always looking down and all I see",
    "Hey, hey Hey, hey, hey, hey Hey, hey, hey, hey Your lipstick stains On the front lobe of my left side brains I knew I wouldn't forget you And so I went and let you blow my mind Your sweet moonbeam The smell of you in every single dream I dream I knew when we collided You're the one I have decided who's one of my kind Hey, soul sister Ain't that Mr. Mister on the radio, stereo The way you move ain't fair, you know Hey, soul sister I don't wanna miss a single thing you do tonight Hey, hey Hey, hey, hey, hey Hey, hey, hey, hey Just in time I'm so glad you have a one-track mind like me You gave my life direction A game show love connection we can't deny I'm so obsessed My heart is bound to beat right out my untrimmed chest I believe in you, like a virgin, you're Madonna And I'm always gonna wanna blow your mind Hey, soul sister Ain't that Mr. Mister on the radio, stereo The way you move ain't fair, you know Hey, soul sister I don't wanna miss a single thing you do tonight The way you can cut a rug Watching you is the only drug I need So gangsta, I'm so thug You're the only one I'm dreaming of You see, I can be myself now finally In fact, there's nothing I can't be I want the world to see you'll be with me Hey, soul sister Ain't that Mr. Mister on the radio, stereo The way you move ain't fair, you know Hey, soul sister I don't wanna miss a single thing you do tonight Hey, soul sister I don't wanna miss a single thing you do tonight Hey, hey Hey, hey, hey, hey Hey, hey, hey, hey (Tonight) Hey, hey Hey, hey, hey, hey Hey, hey, hey, hey (Tonight)",
    "There's a fire starting in my heart Reaching a fever pitch and it's bringing me out the dark Finally, I can see you crystal clear Go ahead and sell me out and I'll lay your ship bare See how I'll leave with every piece of you Don't underestimate the things that I will do There's a fire starting in my heart Reaching a fever pitch and it's bring me out the dark The scars of your love remind me of us They keep me thinking that we almost had it all The scars of your love, they leave me breathless I can't help feeling We could have had it all (You're gonna wish you never had met me) Rolling in the deep (Tears are gonna fall, rolling in the deep) You had my heart inside of your hands (You're gonna wish you never had met me) And you played it to the beat (Tears are gonna fall, rolling in the deep) Baby, I have no story to be told But I've heard one on you and I'm gonna make your head burn Think of me in the depths of your despair Make a home down there as mine sure won't be shared The scars of your love remind me of us (Tears are gonna fall, rolling in the deep) They keep me thinking that we almost had it all (You're gonna wish you never had met me) The scars of your love, they leave me breathless (Tears are gonna fall, rolling in the deep) I can't help feeling We could have had it all (You're gonna wish you never had met me) Rolling in the deep (Tears are gonna fall, rolling in the deep) You had my heart inside of your hands (You're gonna wish you never had met me) And you played it to the beat (Tears are gonna fall, rolling in the deep) Could have had it all Rolling in the deep You had my heart inside of your hands But you played it with a beating Throw your soul through every open door Count your blessings to find what you look for Turn my sorrow into treasured gold You'll pay me back in kind and reap just what you've sown We could have had it all (Tears are gonna fall, rolling in the deep) We could have had it all (You're gonna wish you never had met me) It all, it all, it all (Tears are gonna fall, rolling in the deep) We could have had it all (You're gonna wish you never had met me) Rolling in the deep (Tears are gonna fall, rolling in the deep) You had my heart inside of your hands (You're gonna wish you never had met me) And you played it to the beat (Tears are gonna fall, rolling in the deep) We could have had it all (You're gonna wish you never had met me) Rolling in the deep (Tears are gonna fall, rolling in the deep) You had my heart inside of your hands (You're gonna wish you never had met me) But you played it You played it You played it You played it to the beat",
    "Now and then I think of when we were together Like when you said you felt so happy you could die Told myself that you were right for me But felt so lonely in your company But that was love and it's an ache I still remember You can get addicted to a certain kind of sadness Like resignation to the end, always the end So when we found that we could not make sense Well, you said that we would still be friends But I'll admit that I was glad it was over But you didn't have to cut me off Make out like it never happened and that we were nothing And I don't even need your love But you treat me like a stranger and that feels so rough No, you didn't have to stoop so low Have your friends collect your records And then change your number I guess that I don't need that though Now you're just somebody that I used to know Now you're just somebody that I used to know Now you're just somebody that I used to know Now and then, I think of all the times you screwed me over But had me believing it was always something that I'd done But I don't wanna live that way Reading into every word you say You said that you could let it go And I wouldn't catch, you hung up on Somebody that you used to know But you didn't have to cut me off Make out like it never happened and that we were nothing And I don't even need your love But you treat me like a stranger and that feels so rough And you didn't have to stoop so low Have your friends collect your records And then change your number I guess that I don't need that though Now you're just somebody that I used to know Somebody (I used to know) Somebody (Now you're just somebody that I used to know) Somebody (I used to know) Somebody (Now you're just somebody that I used to know) Somebody",
    "Whoa, oh, oh Whoa, oh, oh Whoa, oh, oh Whoa I'm waking up to ash and dust I wipe my brow and I sweat my rust I'm breathing in the chemicals I'm breaking in, shaping up, then checking out on the prison bus This is it, the apocalypse Whoa I'm waking up, I feel it in my bones Enough to make my systems blow Welcome to the new age, to the new age Welcome to the new age, to the new age Whoa, oh, oh, oh, oh, whoa, oh, oh, oh, I'm radioactive, radioactive Whoa, oh, oh, oh, oh, whoa, oh, oh, oh, I'm radioactive, radioactive I raise my flags, don my clothes It's a revolution, I suppose We'll paint it red to fit right in Whoa I'm breaking in, shaping up, then checking out on the prison bus This is it, the apocalypse Whoa I'm waking up, I feel it in my bones Enough to make my systems blow Welcome to the new age, to the new age Welcome to the new age, to the new age Whoa, oh, oh, oh, oh, whoa, oh, oh, oh, I'm radioactive, radioactive Whoa, oh, oh, oh, oh, whoa, oh, oh, oh, I'm radioactive, radioactive All systems go, the sun hasn't died Deep in my bones, straight from inside I'm waking up, I feel it in my bones Enough to make my systems blow Welcome to the new age, to the new age Welcome to the new age, to the new age Whoa, oh, oh, oh, oh, whoa, oh, oh, oh, I'm radioactive, radioactive Whoa, oh, oh, oh, oh, whoa, oh, oh, oh, I'm radioactive, radioactive",
    "What would I do without your smart mouth? Drawin' me in, and you kickin' me out You've got my head spinnin', no kiddin' I can't pin you down What's going on in that beautiful mind? I'm on your magical mystery ride And I'm so dizzy, don't know what hit me But I'll be alright My head's under water But I'm breathing fine You're crazy and I'm out of my mind 'Cause all of me loves all of you Love your curves and all your edges All your perfect imperfections Give your all to me, I'll give my all to you You're my end and my beginnin' Even when I lose, I'm winnin' 'Cause I give you all of me And you give me all of you, oh-oh How many times do I have to tell you Even when you're crying, you're beautiful too? The world is beating you down I'm around through every mood You're my downfall, you're my muse My worst distraction, my rhythm and blues I can't stop singing, it's ringing in my head for you My head's under water But I'm breathing fine You're crazy, and I'm out of my mind 'Cause all of me loves all of you Love your curves and all your edges All your perfect imperfections Give your all to me, I'll give my all to you You're my end and my beginnin' Even when I lose, I'm winnin' 'Cause I give you all of me And you give me all of you, oh-oh Give me all of you Cards on the table, we're both showing hearts Risking it all, though it's hard 'Cause all of me loves all of you Love your curves and all your edges All your perfect imperfections Give your all to me, I'll give my all to you You're my end and my beginnin' Even when I lose, I'm winnin' 'Cause I give you all of me And you give me all of you I give you all of me And you give me all of you, oh-oh",
    "Doh Doh doh doh, doh doh doh Doh doh Doh doh doh, doh doh doh Doh doh Doh doh doh, doh doh doh Doh doh Aaaaaow! This hit, that ice cold Michelle Pfeiffer, that white gold This one, for them hood girls Them good girls, straight masterpieces Stylin', wildin' Livin' it up in the city Got Chucks on with Saint Laurent Gotta kiss myself, I'm so pretty I'm too hot (hot damn) Called a police and a fireman I'm too hot (hot damn) Make a dragon wanna retire, man I'm too hot (hot damn) Say my name, you know who I am I'm too hot (hot damn) Am I bad 'bout that money? Break it down Girls hit your hallelujah (wooh) Girls hit your hallelujah (wooh) Girls hit your hallelujah (wooh) 'Cause Uptown Funk gon' give it to you (wooh) 'Cause Uptown Funk gon' give it to you 'Cause Uptown Funk gon' give it to you Saturday night, and we in the spot Don't believe me, just watch, come on! Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Hey, hey, hey, oh! Stop Wait a minute Fill my cup, put some liquor in it Take a sip, sign a check Julio, get the stretch Ride to Harlem, Hollywood, Jackson, Mississippi If we show up, we gon' show out Smoother than a fresh jar of Skippy I'm too hot (hot damn) Called a police and a fireman I'm too hot (hot damn) Make a dragon wanna retire, man I'm too hot (hot damn, hot damn) Bitch, say my name, you know who I am I'm too hot (hot damn) Am I bad 'bout that money? Break it down Girls hit your hallelujah (wooh) Girls hit your hallelujah (wooh) Girls hit your hallelujah (wooh) 'Cause Uptown Funk gon' give it to you (wooh) 'Cause Uptown Funk gon' give it to you 'Cause Uptown Funk gon' give it to you Saturday night, and we in the spot Don't believe me, just watch, come on! Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Hey, hey, hey, oh! Before we leave I'mma tell y'all a lil' something Uptown Funk you up Uptown Funk you up Uptown Funk you up Uptown Funk you up I said Uptown Funk you up Uptown Funk you up Uptown Funk you up Uptown Funk you up Come on, dance, jump on it If you sexy than flaunt it If you freaky then own it Don't brag about it, come show me Come on, dance, jump on it If you sexy than flaunt it Well, it's Saturday night, and we in the spot Don't believe me, just watch, come on! Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Don't believe me, just watch Hey, hey, hey, oh! Uptown Funk you up Uptown Funk you up (say what?) Uptown Funk you up Uptown Funk you up Uptown Funk you up Uptown Funk you up (say what?) Uptown Funk you up Uptown Funk you up Uptown Funk you up Uptown Funk you up (say what?) Uptown Funk you up Uptown Funk you up Uptown Funk you up Uptown Funk you up (say what?) Uptown Funk you up",
    "You gotta go and get angry at all of my honesty You know I try but I don't do too well with apologies I hope I don't run out of time, could someone call a referee? Cause I just need one more shot at forgiveness I know you know that I made those mistakes maybe once or twice By once or twice I mean maybe a couple a hundred times So let me, oh let me redeem, oh redeem, oh myself tonight Cause I just need one more shot at second chances Yeah, is it too late now to say sorry? Cause I'm missing more than just your body Is it too late now to say sorry? Yeah I know that I let you down Is it too late to say I'm sorry now? I'm sorry, yeah Sorry, yeah Sorry Yeah I know that I let you down Is it too late to say I'm sorry now? I'll take every single piece of the blame if you want me to But you know that there is no innocent one in this game for two I'll go, I'll go and then you go, you go out and spill the truth Can we both say the words and forget this? Is it too late now to say sorry? Cause I'm missing more than just your body Is it too late now to say sorry? Yeah I know that I let you down Is it too late to say I'm sorry now? I'm not just trying to get you back on me Cause I'm missing more than just your body Is it too late now to say sorry? Yeah I know that I let you down Is it too late to say I'm sorry now? I'm sorry, yeah Sorry, oh Sorry Yeah I know that I let you down Is it too late to say I'm sorry now? I'm sorry, yeah Sorry, oh Sorry Yeah I know that I let you down Is it too late to say I'm sorry now?",
    "It's not good enough for me, since I been with you It's not gonna work for you, nobody can equal me I'm gonna sip on this drink, when I'm fucked up I should know how to pick up I'm gonna catch the rhythm while she push up against me Ooh, and she tipsy I had enough convo for 24 I peep'd you from across the room Pretty little body, dancing like GoGo, aye But you are unforgettable I need to get you alone Why not? A fucking good time, never hurt nobody I got a little drink but it's not Bacardi If you loved the girl then I'm so, so sorry I got to give it to her like we in a marriage Oh, like we in a hurry No, no I won't tell nobody You're on your level too Tryna do what lovers do Feelin' like I'm fresh out, Boosie If they want the drama, got the Uzi Ship the whole crew to the cruise ship Doin' shit you don't even see in movies Ride with me Ride with me, boss I got a hard head but her ass soft She want the last name with the ring on it 'Cause I pulled out a million cash, told her plank on it You are unforgettable I wanna get you alone Now you wanna chose Just popped bubbly in the 'cuzi It's not good enough for me, since I been with you I'm gonna sip on this drink, when I'm fucked up I should know how to pick up I'm gonna catch the rhythm while she push up against me Ooh, and she tipsy I had enough convo for 24 I peep'd you from across the room Pretty little body, dancing like GoGo But you are unforgettable I need to get you alone Why not? A fucking good time, never hurt nobody I got a little drink but it's not Bacardi If you loved your girl then I'm so, so sorry I got to give it to her like we in a marriage Oh, like we in a hurry No, no I won't tell nobody You're on your level too Tryna do what lovers do Oh oh, you ain't enough for me Too much for you Alone Baby go and grab some bad bitches, bring 'em home Know the jet's on me I'ma curve my best for you You know, so pick up that dress for me Leave the rest home Too much combo 24 hours When you stand next to 24 karats She left her man at home She don't love him no more I want your mind and your body Don't mind nobody So as long you don't never hurt nobody Baby girl work your body Work your body And you are unforgettable I need get you alone Now you wanna chose Just popped bubbly in the 'cuzi Right now Oh, like we in a hurry No, no I won't tell nobody You're on your level too Tryna do what lovers do",
    "Sorry if it's hard to catch my vibe, mm-mm I need a lover to trust, tell me you're on my side Are you down for the ride? It's not easy for someone to catch my eye But I've been waitin' for you for my whole damn life For my whole lifetime Don't be afraid to tell me if you ain't with it (you ain't with it) I see you're focused, yeah you're so independent (independent) It's hard for me to open up, I'll admit it (I'll admit it) You've got some shit to say and I'm here to listen So baby, tell me where your love lies Waste the day and spend the night Underneath the sunrise Show me where your love lies I've been so into your mystery Is it because of our history? Are you into me? When it feels so good, but it's bad for you Can't say I don't want it 'cause I know I do Come on over, I need your company Cravin' that synergy Don't be afraid to tell me if you ain't with it (you ain't with it) I see you're focused, yeah, you're so independent (independent) It's hard for me to open up, I'll admit it (I'll admit it) You've got some shit to say and I'm here to listen So baby, tell me where your love lies Waste the day and spend the night Underneath the sunrise Show me where your love lies Tell me where your love lies Waste the day and spend the night Underneath the sunrise Show me where your love lies If you're down, don't hide it Feelin' me, you don't gotta deny it Baby, you gon' make me overnight it Tell me, are you down? If you're down, don't hide it (yeah) Feelin' me, you don't gotta deny it (yeah) Baby, you gon' make me overnight it Tell me, are you down? (Tell me) Tell me where your love lies (tell me where your love lies) Waste the day and spend the night (waste the day, spend the night) Underneath the sunrise (underneath the sunrise) Show me where your love lies (tell me where your love lies) Tell me where your love lies (show me where your love lies) Waste the day and spend the night (yeah, yeah) Underneath the sunrise (ooh) Show me where your love lies",
    "Hmm, hmm I don't wanna be alone tonight (alone tonight) It's pretty clear that I'm not over you (over you, over you) I'm still thinking 'bout the things you do (things you do) So I don't wanna be alone tonight, alone tonight, alone tonight Can you light the fire? (light the fire, light the fire) I need somebody who can take control (take control) I know exactly what I need to do 'Cause I don't wanna be alone tonight, alone tonight, alone tonight Look what you made me do, I'm with somebody new Ooh, baby, baby, I'm dancing with a stranger Look what you made me do, I'm with somebody new Ooh, baby, baby, I'm dancing with a stranger Dancing with a stranger I wasn't even goin' out tonight (out tonight) But, boy, I need to get you off of my mind (off of my mind) I know exactly what I have to do I don't wanna be alone tonight, alone tonight, alone tonight Look what you made me do, I'm with somebody new Ooh, baby, baby, I'm dancing with a stranger Look what you made me do, I'm with somebody new Ooh, baby, baby, I'm dancing with a stranger Dancing with a stranger Dancing with a stranger Dancing, yeah, ooh Look what you made me do (ooh), I'm with somebody new Ooh, baby, baby, I'm dancing with a stranger Look what you made me do, I'm with somebody new Ooh, baby, baby, I'm dancing with a stranger I'm dancing, I'm dancing (ooh) I'm dancing, I'm dancing (dancing with a stranger) I'm dancing, I'm dancing (dancing with a stranger) I'm dancing, I'm dancing (dancing with a stranger)",
    "Yeah I've been tryin' to call I've been on my own for long enough Maybe you can show me how to love, maybe I'm going through withdrawals You don't even have to do too much You can turn me on with just a touch, baby I look around and Sin City's cold and empty (oh) No one's around to judge me (oh) I can't see clearly when you're gone And I said, ooh, I'm blinded by the lights No, I can't sleep until I feel your touch I said, ooh, I'm drowning in the night Oh, when I'm like this, you're the one I trust (Hey, hey, hey) I'm running out of time 'Cause I can see the sun light up the sky So I hit the road in overdrive, baby, oh The city's cold and empty (oh) No one's around to judge me (oh) I can't see clearly when you're gone And I said, ooh, I'm blinded by the lights No, I can't sleep until I feel your touch I said, ooh, I'm drowning in the night Oh, when I'm like this, you're the one I trust I'm just walking by to let you know (by to let you know) I could never say it on the phone (say it on the phone) Will never let you go this time (ooh) I said, ooh, I'm blinded by the lights No, I can't sleep until I feel your touch (Hey, hey, hey) I said, ooh, I'm blinded by the lights No, I can't sleep until I feel your touch"
]

**Note**: Above the lyrics for the rank one songs were manually added to perform sentiment analysis.

In [ ]:
top_song_year = []
top_song_name = []
top_song_appearances = []

for year,songs in by_year.iterrows():
    top = nltk.FreqDist(songs['song'].split('%')).most_common(1)[0]
    top_song_year.append(songs.year)
    top_song_name.append(top[0])
    top_song_appearances.append(top[1])
    
n1_df = pd.DataFrame({'year':top_song_year,'song':top_song_name,'appearances':top_song_appearances,'lyrics':lyrics_of_top_song_n1})



<h1 style="background-color:z;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Preprocessing Lyrics And Feature Engineering</h1>


In [ ]:
n1_df.lyrics = n1_df.lyrics.str.lower() 

#Vader SA
sid = SIA()
n1_df['sentiments']           = n1_df['lyrics'].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x.lower()))))
n1_df['Positive Sentiment']   = n1_df['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
n1_df['Neutral Sentiment']    = n1_df['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
n1_df['Negative Sentiment']   = n1_df['sentiments'].apply(lambda x: x['neg']+1*(10**-6))

n1_df.drop(columns=['sentiments'],inplace=True)

#Number of Words
n1_df['Number_Of_Words'] = n1_df.lyrics.apply(lambda x:len(x.split(' ')))
#Average Word Length
n1_df['Mean_Word_Length'] = n1_df.lyrics.apply(lambda x:np.round(np.mean([len(w) for w in x.split(' ')]),2) )

**Note**: In the above cell, we extract the sentiment from each song lyric by using Vader's lexicon and some naive text attributes.


<h1 style="background-color:z;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Time Based Sentiment Analysis</h1>


In [ ]:
f_data = n1_df.copy()
plt.subplot(2,1,1)
plt.title('Distriubtion Of Sentiments Across Lyrics',fontsize=19,fontweight='bold')
sns.kdeplot(f_data['Negative Sentiment'],label='Negative Sentiment')
sns.kdeplot(f_data['Neutral Sentiment'] ,label='Neutral Sentiment' ,color='orange' )
sns.kdeplot(f_data['Positive Sentiment'],label='Positive Sentiment',color='tab:red')
plt.legend()
plt.subplot(2,1,2)
plt.title('CDF Of Sentiments Across Lyrics',fontsize=19,fontweight='bold')
sns.kdeplot(f_data['Negative Sentiment'],cumulative=True,label='Negative Sentiment')
sns.kdeplot(f_data['Neutral Sentiment'],cumulative=True,label='Neutral Sentiment' ,color='orange' )
sns.kdeplot(f_data['Positive Sentiment'],cumulative=True ,label='Positive Sentiment',color='tab:red')
plt.xlabel('Sentiment Value',fontsize=19)
plt.legend()
plt.show()

**Observation**: we can deduce from the above figure that we have a pretty wide deviation in the neutral sentiment, which is always present in our lyrics, and on average, it represents half of the sentiment of the lyric, i.e., no lyric is mainly negative or positive according to the Vaders lexicon.
Also, on average, the number one songs tend to be more positive than negative.

In [ ]:
f_data=n1_df.copy()
partitions = []
partitions.append(f_data.loc[0:np.round(len(f_data)/3,0)-1,:])
partitions.append(f_data.loc[np.round(len(f_data)/3,0):2*int(len(f_data)/3)-1,:])
partitions.append(f_data.loc[2*np.round(len(f_data)/3,0):3*int(len(f_data)/3)-1,:])



neg_part_means =[]
neg_part_std   =[]
pos_part_means =[]
pos_part_std   =[]
for part in partitions:
    neg_part_means.append(part['Negative Sentiment'].mean())
    neg_part_std.append(part['Negative Sentiment'].std())
    pos_part_means.append(part['Positive Sentiment'].mean())
    pos_part_std.append(part['Positive Sentiment'].std())
    
res_df = pd.DataFrame({'Positive Sentiment Mean':pos_part_means,'Negative Sentiment Mean':neg_part_means,'Positive Sentiment SD':pos_part_std,'Negative Sentiment SD':neg_part_std},
                     index = [f'Partition_{i}' for i in range(1,4)])


def highlight_greater(x):
    temp = x.copy()
    temp = temp.round(0).astype(int)
    m1 = (temp['Partition_1_Mean'] == temp['Partition_2_Mean'])
    m2 = (temp['Partition_1_SD'] == temp['Partition_2_SD'])
    m3 = (temp['Partition_1_Mean'] < temp['Partition_2_Mean']+3) & (temp['Partition_1_Mean'] > temp['Partition_2_Mean']-3)
    m4 = (temp['Partition_1_SD'] < temp['Partition_2_SD']+3) & (temp['Partition_1_SD'] > temp['Partition_2_SD']-3)

    df1 = pd.DataFrame('background-color: ', index=x.index, columns=x.columns)
    #rewrite values by boolean masks
    df1['Partition_1_Mean'] = np.where(~m1, 'background-color: {}'.format('salmon'),        df1['Partition_1_Mean'])
    df1['Partition_2_Mean'] = np.where(~m1, 'background-color: {}'.format('salmon'),        df1['Partition_2_Mean'])
    df1['Partition_1_Mean'] = np.where(m3, 'background-color: {}'.format('gold'),           df1['Partition_1_Mean'])
    df1['Partition_2_Mean'] = np.where(m3, 'background-color: {}'.format('gold'),           df1['Partition_2_Mean'])
    df1['Partition_1_Mean'] = np.where(m1, 'background-color: {}'.format('mediumseagreen'), df1['Partition_1_Mean'])
    df1['Partition_2_Mean'] = np.where(m1, 'background-color: {}'.format('mediumseagreen'), df1['Partition_2_Mean'])

    df1['Partition_1_SD'] = np.where(~m2, 'background-color: {}'.format('salmon'),        df1['Partition_1_SD'])
    df1['Partition_2_SD'] = np.where(~m2, 'background-color: {}'.format('salmon'),        df1['Partition_2_SD'])
    df1['Partition_1_SD'] = np.where(m4, 'background-color: {}'.format('gold'),           df1['Partition_1_SD'])
    df1['Partition_2_SD'] = np.where(m4, 'background-color: {}'.format('gold'),           df1['Partition_2_SD'])
    df1['Partition_1_SD'] = np.where(m2, 'background-color: {}'.format('mediumseagreen'), df1['Partition_1_SD'])
    df1['Partition_2_SD'] = np.where(m2, 'background-color: {}'.format('mediumseagreen'), df1['Partition_2_SD'])

    return df1



#res_df.style.apply(highlight_greater,axis=None)
res_df = res_df.T
res_df = pd.DataFrame(res_df.values,columns=res_df.columns,index=['Positive Sentiment','Negative Sentiment','Positive Sentiment','Negative Sentiment'])
res_df = pd.concat([res_df.iloc[:2,:],res_df.iloc[2:,:]],axis=1)
res_df.columns = ['Partition_1_Mean','Partition_2_Mean','Partition_3_Mean','Partition_1_SD','Partition_2_SD','Partition_3_SD']
res_df.style.apply(highlight_greater,axis=None)

In [ ]:
fig = make_subplots(rows=3, cols=2)

for idx,prt in enumerate(partitions):
    fig.add_trace(
    go.Scatter(x=prt['year'], y=prt['Positive Sentiment'],name=f'Positive Part {idx+1}'),
    row=idx+1, col=1)
    fig.add_trace(
    go.Scatter(x=prt['year'], y=prt['Negative Sentiment'],name=f'Negative Part {idx+1}'),
    row=idx+1, col=2)

fig.update_layout(height=600, width=900, title_text="Distibution Of Daily Sentiments Over Our Time Line For Each Partition")
fig.show()

**Note**: Above is a small segment of our test for sentiment stationarity over our timeline, and it seems that the mean and standard deviation in all three of our partitions has no significant difference, which may point to a non-constant mean/variance.

In [ ]:
ft_data=f_data.copy()
fig = make_subplots(rows=4, cols=2, subplot_titles=('Observed Pos', 'Observed Neg', 'Trend Pos','Trend Neg','Seasonal Pos','Seasonal Neg','Residual Pos','Residual Neg'))
b_date_mean = ft_data.groupby(by='year').mean().reset_index()

lbl = ['Positive','Negative']

for idx,column in enumerate(['Positive Sentiment','Negative Sentiment']):
    res = seasonal_decompose(b_date_mean[column], period=10, model='additive', extrapolate_trend='freq')
    
    fig.add_trace(
    go.Scatter(x=n1_df.year, y=res.observed,name='{} Observed'.format(lbl[idx])),
    row=1, col=idx+1)
    
    fig.add_trace(
    go.Scatter(x=n1_df.year, y=res.trend,name='{} Trend'.format(lbl[idx])),
    row=2, col=idx+1)
    
    fig.add_trace(
    go.Scatter(x=n1_df.year, y=res.seasonal,name='{} Seasonal'.format(lbl[idx])),
    row=3, col=idx+1)
    
    fig.add_trace(
    go.Scatter(x=n1_df.year, y=res.resid,name='{} Residual'.format(lbl[idx])),
    row=4, col=idx+1)
            
fig.update_layout(height=600, width=900, title_text="Decomposition Of Our Sentiments into Trend,Level,Seasonality and Residuals")
fig.show()

**Observation**: After decomposing both the positive and negative sentiments using a ten-year period, we can see a rising trend in negativity in the rank one lyrics and previous positivity trend that peaked in 1978 and declined from there.

In [ ]:
f, ax = plt.subplots(nrows=2, ncols=2, figsize=(16, 10))
ax[0,0].set_ylim(-1.1,1.1)
ax[1,0].set_ylim(-1.1,1.1)
ax[0,1].set_ylim(-1.1,1.1)
ax[1,1].set_ylim(-1.1,1.1)

plot_acf(b_date_mean['Negative Sentiment'],lags=20, ax=ax[0,0],title='Autocorrelation Negative')
plot_pacf(b_date_mean['Negative Sentiment'],lags=20, ax=ax[1,0],title='Partial Autocorrelation Negative')
plot_acf(b_date_mean['Positive Sentiment'],lags=20, ax=ax[0,1],color='tab:red',title='Autocorrelation Positive')
plot_pacf(b_date_mean['Positive Sentiment'],lags=20, ax=ax[1,1],color='tab:red',title='Partial Autocorrelation Positive')
plt.show()

**Observation**: It seems that we do not have any significant autocorrelation or partial autocorrelation between any of our lags, with an exception for a single relatively small correlation between the positive sentiment and its lag 1.


<h1 style="background-color:z;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Similarity Analysis</h1>


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance_matrix
lyr = n1_df.lyrics.copy()
lyr = lyr.apply(lambda x:' '.join(re.findall(r'\w+', x)))
#lyr = lyr.apply(lambda x:x.replace(',',''))


tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,1))
tf_df = tfidf.fit_transform(lyr)
tf_df = pd.DataFrame(tf_df.toarray(),columns=tfidf.vocabulary_,index=n1_df.year)

distances = []
years = []
for i in range(0,len(tf_df)-1):
    distance = 1-(tf_df.iloc[i,:]@tf_df.iloc[i+1,:])/(np.linalg.norm(tf_df.iloc[i,:])*np.linalg.norm(tf_df.iloc[i+1,:]))
    years.append(str(tf_df.index[i])+'-'+str(tf_df.index[i+1]))
    distances.append(distance)
    
fig = go.Figure()
trace1 = go.Scatter(x=years,y=distances,mode='lines+markers',name='Cosine Similarity',showlegend=True)

fig.add_trace(trace1)
fig.add_vrect(x0="1978-1979", x1="1993-1994", 
              annotation_text="$Period_{1}$", annotation_position="top left",
              fillcolor="green", opacity=0.25, line_width=0)

fig.add_vrect(x0="2012-2013", x1="2019-2020", 
              annotation_text="$Period_{2}$", annotation_position="top left",
              fillcolor="orange", opacity=0.25, line_width=0)

fig.update_layout(hovermode='x unified',yaxis_title='Similarity',xaxis_title='Consecutive Years',title='Cosine Similarity of Each Song With the Next Years Song')
fig.show()

Observation: In the above figure shown the cosine similarity between the lyrics of each consecutive two years.
Our point of interest is seeing if there are any periods on our timeline in which the song lyrics were significantly different, and we indeed see such behavior.
The two periods with the most significant deviation between consecutive years are colored in green and orange in the figure.


In [ ]:
NUMBER_OF_COMPONENTS = 63

svd = TruncatedSVD(NUMBER_OF_COMPONENTS)
decomposed = svd.fit_transform(tf_df)

evr = svd.explained_variance_ratio_
total_var = evr.sum() * 100
cumsum_evr = np.cumsum(evr)

trace1 = {
    "name": "individual explained variance", 
    "type": "bar", 
    'y':evr}
trace2 = {
    "name": "cumulative explained variance", 
    "type": "scatter", 
     'y':cumsum_evr}
data = [trace1, trace2]
layout = {
    "xaxis": {"title": "Principal components"}, 
    "yaxis": {"title": "Explained variance ratio"},
  }
fig = go.Figure(data=data, layout=layout)
fig.update_layout(     title='{:.2f}% of the Song Lyrics Variance Can Be Explained Using {} Words'.format(np.sum(evr)*100,NUMBER_OF_COMPONENTS))
fig.show()

In [ ]:
best_fearures = [[tfidf.get_feature_names()[i],svd.components_[0][i]] for i in svd.components_[0].argsort()[::-1]]
worddf = pd.DataFrame(np.array(best_fearures[:63])[:,0]).rename(columns={0:'Word'})
worddf['Explained Variance'] =  np.round(evr*100,2)
worddf['Explained Variance'] =worddf['Explained Variance'].apply(lambda x:str(x)+'%')
app = []
for word in worddf.Word:
    total_count = 0
    for text in lyr:
        if text.find(word)!= -1:
            total_count+=1
    app.append(total_count)
worddf['Appeared_On_X_Tweets'] = app
worddf

fig = go.Figure()
fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Word<b>',"<b>Accountable for X% of Variance<b>",'<b>Appeared On X Songs<b>'],
            font=dict(size=19,family="Lato"),
            align="center"
        ),
        cells=dict(
            values=[worddf[k].tolist() for k in ['Word',"Explained Variance",'Appeared_On_X_Tweets']],
            align = "center")
    ),
    
)

fig.show()

**Observation**: A quite surprising and outstanding discovery is that all the song lyrics' variance can be described using only 63 words!

In [ ]:
dm = distance_matrix(decomposed,decomposed,p=1)
dm = np.round(dm,2)
dm = (dm-dm.min())/(dm.max()-dm.min())
# fig = ff.create_annotated_heatmap(z=dm, colorscale='Viridis')
# fig.show()

sns.clustermap(dm,cmap='vlag')

**Observation**: The above cluster map shows the segmentation of the song lyrics into clusters based on the distances between them, and we see that although there is no segmentation into a large cluster, there are many similar songs that arise questions like, how original are the lyrics of the most popular song in the Billboard top 100 and how much did our taste in music changed in 63 years? Is there some formation for the perfect catchy song?

<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Artist Analysis</h1>


In [ ]:
by_year = b_df.groupby("year")

by_year = by_year["artist"].agg(lambda column: "%".join(column)).to_frame().reset_index()


n_song_year                                                       = []
n1_artist_name,n2_artist_name,n3_artist_name                      = [],[],[]
n1_artist_appearances,n2_artist_appearances,n3_artist_appearances = [],[],[]

for year,songs in by_year.iterrows():
    top = nltk.FreqDist(songs['artist'].split('%')).most_common(3)
    n_song_year.append(songs.year)
    n1_artist_name.append(top[0][0])
    n2_artist_name.append(top[1][0])
    n3_artist_name.append(top[2][0])
    n1_artist_appearances.append(top[0][1])
    n2_artist_appearances.append(top[1][1])
    n3_artist_appearances.append(top[2][1])

    


fig = go.Figure()

n1 = go.Scatter(x=n_song_year,y=n1_artist_appearances,mode='lines+markers',text =[f"Song Name: <b>{i}<b>" for i in n1_artist_name],showlegend=True,name='<b>№ 1 Artist<b>')  
n2 = go.Scatter(x=n_song_year,y=n2_artist_appearances,mode='lines+markers',text =[f"Song Name: <b>{i}<b>" for i in n2_artist_name],showlegend=True,name='<b>№ 2 Artist<b>')  
n3 = go.Scatter(x=n_song_year,y=n3_artist_appearances,mode='lines+markers',text =[f"Song Name: <b>{i}<b>" for i in n3_artist_name],showlegend=True,name='<b>№ 3 Artist<b>')


fig.add_trace(n1)
fig.add_trace(n2)
fig.add_trace(n3)

fig.update_layout(title='<b>Yearly Top Artist and The Number of Appearances on Lists<b>',xaxis_title='<b>Year<b>',yaxis_title='<b>Numer of Appearances<b>')
fig.update_layout(hovermode="x unified")

fig.show()


In [ ]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=('<b>Difference Between № 1 and № 2 in a Given Year<b>',  '<b>Difference Between № 2 and № 3 in a Given Year<b>'))
n1_np = np.array(n1_artist_appearances)
n2_np = np.array(n2_artist_appearances)
n3_np = np.array(n3_artist_appearances)

    
R = pd.DataFrame({'Artist':np.array(n1_artist_name)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()],
'Appearance':np.array(n1_artist_appearances)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()],
              'Year':np.array(n_song_year)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()]})

outstanding_artists = R.copy()

fig.add_trace(
    go.Scatter(x=n_song_year, y=np.abs(n1_np-n2_np),name='Difference'),
    row=1, col=1
)
fig.add_shape(type="line",
    x0=n_song_year[0], y0=np.abs(n1_np-n2_np).mean()+np.abs(n1_np-n2_np).std(), x1=n_song_year[-1], y1=np.abs(n1_np-n2_np).mean()+np.abs(n1_np-n2_np).std(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x1', 
        yref='y1')

fig.add_trace(
    go.Scatter(x=n_song_year, y=np.abs(n2_np-n3_np),name='Difference'),
    row=2, col=1
)
fig.add_shape(type="line",
    x0=n_song_year[0], y0=np.abs(n2_np-n3_np).mean()+np.abs(n2_np-n3_np).std(), x1=n_song_year[-1], y1=np.abs(n2_np-n3_np).mean()+np.abs(n2_np-n3_np).std(),
    line=dict(
        color="Red",
        width=2,
        dash="dashdot",
    ),
        name='Mean',
        xref='x1', 
        yref='y2')


fig['layout']['xaxis2']['title'] = 'Date'
fig['layout']['yaxis2']['title'] = 'Appearances Difference'
fig['layout']['yaxis1']['title'] = 'Appearances Difference'

fig.update_layout(title_text="Difference in Artist Appearances Between Ranks")
fig.update_layout(hovermode="x unified")

fig.show()

In [ ]:
R = pd.DataFrame({'Artist':np.array(n1_artist_name)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()],
'Appearance':np.array(n1_artist_appearances)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()],
              'Year':np.array(n_song_year)[(n1_np-n2_np)>(n1_np-n2_np).mean()+(n1_np-n2_np).std()]})

fig = go.Figure()
fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Artist<b>',"<b>Appearance<b>",'<b>Year<b>'],
            font=dict(size=19,family="Lato"),
            align="center"
        ),
        cells=dict(
            values=[R[k].tolist() for k in ['Artist',"Appearance",'Year']],
            align = "center")
    ),
    
)

fig.update_layout(title='List of Artists that Were the Most Significantly Different From 2 and 3 Place')
fig.show()

**Observation**: Similarly to the analysis we performed on the top-ranking songs that we based on the number of appearances in a particular year, the same method was applied to analyze the artists.
Interestingly, observing outstandingly famous artists in a particular year i.e., having a number of appearances which is above the average plus one standard deviation, is much rarer.
So rare that only six artists ever achieved such a position.
Another important observation is that those rare artists' years seem to become more frequent as we move towards the present.

<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">Probabilistic Analysis</h1>


In [ ]:
os_song_freq = n1_df.year.to_frame()
os_artist_freq = n1_df.year.to_frame()
os_song_freq['outstanding']=os_song_freq.year.apply(lambda x: 1 if x in outstanding_songs.Year.to_list() else 0)
os_artist_freq['outstanding']=os_artist_freq.year.apply(lambda x: 1 if x in outstanding_artists.Year.to_list() else 0)


fig = make_subplots(rows=2, cols=1,shared_xaxes=True,subplot_titles=('<b>Outstanding Song Occurrence Over Our Timeline<b>',  '<b>Outstanding Artist Occurrence Over Our Timeline<b>'))

fig.add_trace(go.Scatter(x=os_song_freq.year,y=os_song_freq.outstanding,mode='markers',name='song'),
             row=1,
             col=1)

fig.add_trace(go.Scatter(x=os_artist_freq.year,y=os_artist_freq.outstanding,mode='markers',name='Artist'),
             row=2,
             col=1)
#ex.scatter(os_song_freq,x='year',y='outstanding',title='Outstanding Song Occurrence Over Our Timeline')

To estimate the probability of a song being "outstanding," i.e., be the number one most appeared song in its years and have a significant difference between the second rank, we will use Bayesian inference.
We will estimate our prior probability for a sing being "outstanding," using a uniform prior as we do not have any prior knowledge or belief regarding such songs' rate.
We define $$\Theta \sim Uniform(0,1)$$ to be prior for the P parameter that we will sample.
The process of a song being "outstanding" can be modeled by using a Bernoulli distribution with a parameter sampled from our uniform prior, i.e., $$Outstanding \sim Ber(\Theta)$$


In [ ]:
import pymc3 as pm
with pm.Model():
    theta = pm.Uniform('P',0,1)
    obs = pm.Bernoulli('obs',p=theta,observed = os_song_freq.outstanding)
    
    N = 50000

    step = pm.Metropolis()
    trace = pm.sample(N,step=step)
    burned_trace = trace[N-int(N*0.9):]

In [ ]:
beta_samples = burned_trace['P']

plt.subplot(211)
plt.title(r"Posterior distributions of the Probability Of A Outstanding Song After Observing Our Data")
_hist = plt.hist(beta_samples, histtype='stepfilled', bins=35, alpha=0.85,
         label=r"posterior of $P_{Outstanding}$", color="#7A68A6")
plt.vlines(beta_samples.mean(),0,1.1*np.max(_hist[0]),label='Posterior Mean',linestyle='-.',linewidth=3,color='red')
plt.vlines(beta_samples.mean()-2*beta_samples.std(),0,1.1*np.max(_hist[0]),label='95% Confidence Bound',linestyle='-.',linewidth=3,color='tab:green')
plt.vlines(beta_samples.mean()+2*beta_samples.std(),0,1.1*np.max(_hist[0]),linestyle='-.',linewidth=3,color='tab:green')

plt.legend(prop=dict(size=20))
plt.show()

Observation: After sampling from our posterior space using Metropolis MCMC we see that our posterior distribution is quite wide due to the small number of observations, answering our hypothesis with a wide range of possible value of $\Theta$.
Still, we can say with high confidence that the probability of a random song from the number one list being "outstanding" is between 0.07 to 0.27, centered around 0.17.

In [ ]:
with pm.Model():
    P_Song    = pm.Uniform('P_S',0,1)
    P_Artist  = pm.Uniform('P_A',0,1)
    
    Diff = pm.Deterministic('delta',P_Song-P_Artist)
    
    obs_a = pm.Bernoulli('obs_S',p=P_Song,observed = os_song_freq.outstanding)
    obs_b = pm.Bernoulli('obs_A',p=P_Artist,observed = os_artist_freq.outstanding)

    N = 50000

    step = pm.Metropolis()
    trace = pm.sample(N,step=step)
    burned_trace = trace[N-int(N*0.9):]

In [ ]:

p_A_samples=burned_trace['P_S']
p_B_samples=burned_trace['P_A']
delta_samples=burned_trace['delta']

ax = plt.subplot(311) 
plt.xlim(0, .4) 
plt.hist(p_A_samples, histtype='stepfilled', bins=25, alpha=0.85, label="posterior of $P_{song}$", color="#A60628")
#plt.vlines(true_p_A, 0, 80, linestyle="--", label="true $p_A$ (unknown)")
plt.legend(loc="upper right",prop=dict(size=15))
plt.title("Posterior distributions of $P_{song}$, $P_{artist}$, and delta unknowns",fontsize=16) 
ax = plt.subplot(312) 
plt.xlim(0, .4)
plt.hist(p_B_samples, histtype='stepfilled', bins=25, alpha=0.85, label="posterior of $P_{artist}$", color="#467821")
#plt.vlines(true_p_B, 0, 80, linestyle="--", label="true $p_B$ (unknown)")
plt.legend(loc="upper right",prop=dict(size=15)) 
ax = plt.subplot(313)
plt.hist(delta_samples, histtype='stepfilled', bins=30, alpha=0.85, label="posterior of delta", color="#7A68A6")
#plt.vlines(true_p_A - true_p_B, 0, 60, linestyle="--", label="true delta (unknown)")
plt.vlines(0, 0, 60, color="black", alpha=0.2)
plt.legend(loc="upper right",prop=dict(size=15)); 


<a id="1"></a>

<h1 style="background-color:orange;font-family:newtimeroman;font-size:170%;text-align:center;border-radius: 15px 50px;">"Outstanding" Song Lyrics Decomposition</h1>


In [ ]:
ot_df =  n1_df[os_song_freq.outstanding==1].copy()
wc = WordCloud(width=800,height=300,background_color='white',collocations = False,stopwords=STOPWORDS).generate(' '.join(ot_df.lyrics))

plt.title('Most Common Words in Outstanding Songs Lyrics',fontsize=16)
plt.imshow(wc)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
exclude = set(string.punctuation)
ot_df.lyrics = ot_df.lyrics.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

tokenized_text = nltk.word_tokenize(' '.join(ot_df.lyrics))
tokenized_text = [word for word in tokenized_text if word not in STOPWORDS and word not in string.punctuation and len(word)>2]
freqD = nltk.FreqDist(tokenized_text)

top_10_words = list(dict(freqD.most_common(10)).keys())
top_10_words


ag_june = nltk.FreqDist(tokenized_text.copy())

token=tokenized_text.copy()
june_bigram=ngrams(tokenized_text,2)
june_trigram=ngrams(tokenized_text,3)

june_bdict = dict()
for i in list(june_bigram):
    june_bdict[i] = june_bdict.get(i,0)+1
    
june_bdict = {k: v for k, v in sorted(june_bdict.items(), key=lambda item: item[1],reverse=True)}


june_tridict = dict()
for i in list(june_trigram):
    june_tridict[i] = june_tridict.get(i,0)+1
    
june_tridict = {k: v for k, v in sorted(june_tridict.items(), key=lambda item: item[1],reverse=True)}

def calculate_tri_prob(tri,firt_order,second_order,third_order):
    fw,sw,tw = tri
    p_fw=firt_order[fw]
    p_fwsw = second_order[(fw,sw)]/p_fw
    p_fwswtw = third_order[(fw,sw,tw)]/p_fw
    return p_fwswtw

def calculate_sentence_prob(sentence,firt_order,second_order):
    p_first = firt_order[sentence[0]]/len(firt_order.keys())
    p_second_first = second_order[(sentence[0],sentence[1])]/len(second_order)
    p_thid_second = second_order[(sentence[1],sentence[2])]/len(second_order)
    return p_first*p_second_first*p_thid_second

top_10_tri_june = []

for key in list(june_tridict.keys())[:15]:
    top_10_tri_june.append((key,calculate_tri_prob(key,ag_june,june_bdict,june_tridict)))
    
    
june_tri_df = pd.DataFrame(top_10_tri_june,columns=['Sentence','Probability Given First Words'])

june_tri_df['First Word'] = june_tri_df.Sentence.apply(lambda x: x[0])
june_tri_df['Sentence Probability-Markov Assumption'] = june_tri_df.Sentence.apply(lambda x: calculate_sentence_prob(x,ag_june,june_bdict))
june_tri_df = june_tri_df[['First Word','Sentence','Probability Given First Words','Sentence Probability-Markov Assumption']]
june_tri_df.Sentence=june_tri_df.Sentence.apply(lambda x: ' '.join(x))
june_tri_df.style.background_gradient(cmap='coolwarm')

In [ ]:
import networkx as nx
lyrics_bigram=ngrams(tokenized_text,2)
lyrics_bigram = list(lyrics_bigram)

freq_b = nltk.FreqDist(lyrics_bigram)
edges = list(dict(freq_b).keys())
edges = [[x,y] for x,y in edges]
nodes = np.unique(np.array(edges).flatten())
N_nodes = len(nodes)
G = nx.Graph()
G.add_nodes_from(nodes)
#pos = nx.drawing.nx_pydot.graphviz_layout(G)
for x,y in edges:
    G.add_edge(x,y,value=freq_b[(x,y)])

G = nx.DiGraph(G)
pos = nx.spiral_layout(G,resolution =3.5)

node_x = []
node_y = []
weights = []
for node in pos:
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

for E in edges:
    weights.append(freq_b[tuple(E)])
weights = np.array(weights)
#weights = (weights-weights.min())/(weights.max()-weights.min())

In [ ]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

#EDGE TRACE
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.9, color='#888'),
    hoverinfo='none',
    mode='lines')
#NODE TRACE

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Jet',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

#ADJ COUNT
node_text = []
node_adjacencies = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])) +' <b>| Word: [<b>' +list(G.nodes())[node]+']')

#modify NODE TRACE
node_trace.marker.color = node_adjacencies
node_trace.text = node_text

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<b>Lyrics Bigram Word Network<b>',
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

In [ ]:
from pyvis.network import Network

nt = Network('500px', '900px',notebook=True)
nt.set_options('var options = { "physics": {"forceAtlas2Based": {"gravitationalConstant": -230,"springLength": 170,\
      "springConstant": 0,\
      "avoidOverlap": 1\
    },\
    "minVelocity": 0.75,\
    "solver": "forceAtlas2Based",\
    "timestep": 1\
  }\
}\
               ')
nt.from_nx(G)
#nt.show_buttons(filter_=['physics'])
nt.prep_notebook()
nt.show('nx.html')
